# ***Génération de Données Synthétiques de Ventes***
---

In [ ]:
import pandas as pd
df=pd.read_csv('features.csv')

# Boissons Fraîches

Ce code permet de générer des données synthétiques réalistes de ventes de boissons fraîches en intégrant plusieurs facteurs d'influence :

- **Météorologie** (température, précipitations, humidité, vent)
- **Effets régionaux** (zones côtières, densité urbaine)
- **Jours spéciaux** (weekends, fériés, Ramadan, Nouvel An)
- **Bruit aléatoire** pour simuler des fluctuations naturelles

## 1. Coefficients Régionaux et Jours Spéciaux
**region_boisson_coefficients** & **special_day_coefficients**  
**Objectif :** Modéliser des variations géographiques et temporelles réalistes.

### Méthodologie :

- **Régions :** Les zones côtières (Côtière) ont un coefficient élevé (+70%) pour refléter une demande touristique accrue.
- **Jours spéciaux :**
  - Ramadan et New Year ont les coefficients les plus forts (respectivement +70% et +80%) pour simuler des pics de consommation culturels.
  - Les weekends (+30%) et jours fériés (+20%) suivent des tendances connues en distribution.


## 2. Impact Météorologique (meteo_boisson_impact)

### Effets Climatiques

- **Température (temp_effect) :**
  - Base : 50% des ventes en dessous de 10°C (faible demande).
  - Gradient : +2% par °C entre 10°C et 20°C, puis +4% entre 20°C et 30°C (effet non linéaire).
  - Plafond : Limité à +50% au-delà de 30°C pour éviter des valeurs irréalistes.

- **Précipitations (precip_effect) :**  
  Réduction linéaire (jusqu’à -60% pour de fortes pluies), avec un minimum de 40% des ventes.

- **Vent (wind_effect) :**  
  Pénalité de -5% uniquement si >25°C et vent >20 km/h (effet combiné chaleur/vent).

- **Humidité (humidity_effect) :**  
  Augmentation de +1% par point d’humidité >50% si chaud (>20°C), légère diminution sinon.

### Équation Combinée :

    impact_final = max(0.3, temp_effect × precip_effect × wind_effect × humidity_effect)
→ Le minimum de 30% garantit une demande résiduelle même par mauvais temps.

## 3. Génération et Ajustement des Ventes (generer_ventes_boissonsfraiches & ajuster_ventes)

### Étapes Clés

- **Ventes de Base :**  
  Calculées via `calcul_impact_climatique()` appliqué à chaque ligne du DataFrame.  
  Base arbitraire de 50 unités (ajustable via `base_ventes`).

- **Ajustements Supplémentaires :**
  - **Région :** Coefficients multipliés par 1.2 en haute saison touristique.
  - **Jours Spéciaux :**  
    Gestion des cumuls (ex: un jour férié tombant un weekend prend le coefficient max + 10%).
    Ramadan/New Year : Bonus supplémentaires (+20% et +50%) pour capturer des pics extrêmes.

- **Tendances :**  
  Hausse de +30% si réchauffement brutal (>+5°C par rapport à la veille).

- **Densité Urbaine :**  
  Métropoles (+50%) > Villes moyennes (+20%) > Petites villes (neutre).

- **Limites Réalistes :**  
  Plafonds de ventes (`max_ventes_jour_normal` et `max_ventes_jour_special`) pour éviter des outliers non physiques.

## 4. Ajout de Bruit (ajouter_bruit)

- **Distribution :** Bruit multiplicatif suivant une loi normale (μ=1.0, σ=0.08).
- **Post-traitement :**
  - Valeurs forcées à ≥1 (pas de ventes négatives).
  - Arrondi à l’entier pour imiter des comptages réels.

**But :** Simuler des variations aléatoires (ex: promotions locales, événements imprévus).

## 5. Validation (print_statistiques)

### Contrôles :

- Comparaison des moyennes (Ramadan vs hors Ramadan, weekends vs semaine, etc.).
- Analyse par région (`profil_cotier`) pour vérifier la cohérence des coefficients.



In [ ]:
import numpy as np 
# Coefficients régionaux
region_boisson_coefficients = {
    'Côtière': 1.7,     # +70% pour les zones touristiques
    'Hors côtier': 1.5,  # +50% plus de population
    'Intérieure': 1.0,     # pas d'impact
}

special_day_coefficients = {
    'Weekend': 1.3,    # +30%
    'jour_ferie': 1.2, # +20%
    'vacances_scolaires': 1.1,  # léger impact positif
    'ramadan': 1.4,    # réduire de 1.7 à 1.4 (+40% au lieu de +70%)
    'new_year': 1.5    # réduire de 1.8 à 1.5 (+50% au lieu de +80%)
}

max_ventes_jour_normal=25
max_ventes_jour_special=35
def meteo_boisson_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    if temp_jour < 10:
        temp_effect = 0.5  # 50% des ventes normales en dessous de 10°C
    elif 10 <= temp_jour < 20:
        temp_effect = 0.5 + 0.03 * (temp_jour - 10)  # +3% par °C (augmenté de 2% à 3%)
    elif 20 <= temp_jour < 30:
        temp_effect = 0.8 + 0.05 * (temp_jour - 20)  # +5% par °C (augmenté de 4% à 5%)
    else:
        temp_effect = 1.3 + 0.02 * (temp_jour - 30)  # Base augmentée de 1.1 à 1.3
        temp_effect = min(temp_effect, 1.7)  # Plafonné à +70% (augmenté de 50% à 70%)
    # Effet précipitations (réduit les ventes)
    precip_effect = max(0.4, 1 - (precip_jour * 0.08))  # Effet réduit et minimum 40% des ventes normales

    # Effet vent (réduit légèrement les ventes quand il fait chaud)
    wind_effect = 1.0
    if temp_jour > 25 and wind_speed_jour > 20:
        wind_effect = 0.95  # -5% si chaud et venteux

    # Effet humidité
    if temp_jour >= 20:
        # Augmentation de 1% pour chaque point d'humidité au-dessus de 50%
        humidity_effect = 1 + 0.01 * max(0, humidity_jour - 50)
    else:
        # Réduction moins importante
        humidity_effect = 1 - 0.003 * max(0, humidity_jour - 50)

    # Combinaison des effets
    return max(0.3, temp_effect * precip_effect * wind_effect * humidity_effect)  # Minimum 30% des ventes de base

def calcul_impact_climatique(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    # Calcul de l'impact climatique
    impact = meteo_boisson_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)
    return impact

def generer_ventes_boissonsfraiches(df, base_ventes=30):
    """Génère les ventes avec impact climatique de base puis ajustements conditionnels."""


    df['boissons fraiches'] = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

def ajuster_ventes(df):
    """Ajuste les ventes de base en fonction des autres features."""



    # Coefficients régionaux
    df['region_coeff'] = df['profil_cotier'].map(region_boisson_coefficients)

    # Bonus pour les zones touristiques
    df.loc[df['saison_toristique'] == 'Haute', 'region_coeff'] *= 1.2

    # Coefficients pour jours spéciaux
    df['special_coeff'] = 1.0

    # Appliquer les coefficients de jour spécial séparément et non cumulativement
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
        mask = df[day_type] == 1
        if day_type in special_day_coefficients and mask.any():
            df.loc[mask, 'special_coeff'] = special_day_coefficients[day_type]

    # Pour les jours avec plusieurs types (exemple: weekend + jour férié), prendre le coefficient le plus élevé
    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            df.loc[both_mask, 'special_coeff'] = max(
                special_day_coefficients['jour_ferie'],
                special_day_coefficients['Weekend']
            ) * 1.1  # Bonus de 10% pour combinaison

    # Effet de tendance (variations par rapport à la veille)
    df['trend_effect'] = 1.0
    if 'temp_jour_lag1' in df.columns:
        delta_temp = df['temp_jour'] - df['temp_jour_lag1']
        df.loc[(delta_temp > 5) & (df['temp_jour'] > 25), 'trend_effect'] = 1.3

    # Effet de la densité urbaine
    densite_map = {
        'Grande métropole': 1.5,
        'Ville moyenne': 1.2,
        'Petite ville': 1.0
    }
    df['densite_effect'] = df['densite_urbaine'].map(densite_map)

    # Effet de la région climatique
    climat_map = {
        'Méditerranéenne': 1.2,
        'Semi-aride': 1.3,
        'Aride/désertique': 1.4
    }
    df['climat_effect'] = df['region_climatique'].map(climat_map)

    # Augmenter l'effet Ramadan et New Year
    df.loc[df['ramadan'] == 1, 'special_coeff'] = special_day_coefficients['ramadan'] * 1.2  # +20% supplémentaire
    df.loc[df['new_year'] == 1, 'special_coeff'] = special_day_coefficients['new_year'] * 1.5  # +50% supplémentaire

    # Calcul final des ventes ajustées
    df['boissons fraiches'] = df['boissons fraiches'] * df['region_coeff'] * df['special_coeff'] * df['trend_effect'] * df['densite_effect'] * df['climat_effect']

    # Limitation des ventes journalières
    # Jours normaux
    df.loc[
        (~df['ramadan'].astype(bool)) & (~df['new_year'].astype(bool)) & (df['boissons fraiches'] > max_ventes_jour_normal),
        'boissons fraiches'
    ] = max_ventes_jour_normal

    # Jours spéciaux (Ramadan, New Year)
    df.loc[
        (df['ramadan'].astype(bool) | df['new_year'].astype(bool)),
        'boissons fraiches'
    ] = df.loc[
        (df['ramadan'].astype(bool) | df['new_year'].astype(bool)),
        'boissons fraiches'
    ].clip(upper=max_ventes_jour_special)

    return df

# Génération de bruit
def ajouter_bruit(df, niveau_bruit=0.05):  # Réduire de 0.08 à 0.05
    # Utiliser un bruit plus cohérent
    np.random.seed(42)  # Pour la reproductibilité
    
    # Ajouter de l'autocorrélation au bruit
    n = len(df)
    bruit_base = np.random.normal(0, niveau_bruit, size=n)
    bruit_autocorrelé = np.zeros(n)
    
    # Faire dépendre chaque valeur partiellement des valeurs précédentes
    for i in range(n):
        if i == 0:
            bruit_autocorrelé[i] = bruit_base[i]
        else:
            bruit_autocorrelé[i] = 0.7 * bruit_autocorrelé[i-1] + 0.3 * bruit_base[i]
    
    facteurs_multiplicatifs = 1.0 + bruit_autocorrelé
    
    df_avec_bruit = df.copy()
    df_avec_bruit["boissons fraiches"] = df["boissons fraiches"] * facteurs_multiplicatifs
    df_avec_bruit["boissons fraiches"] = np.maximum(1, df_avec_bruit["boissons fraiches"])

    # Arrondir aux entiers
    df_avec_bruit["boissons fraiches"] = df_avec_bruit["boissons fraiches"].round().astype(int)
    return df_avec_bruit
def processus_de_generation_complet(df, base_ventes=30, niveau_bruit=0.05):

    if df is None:
        raise ValueError("Input DataFrame cannot be None")
    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_boissonsfraiches(df, base_ventes=base_ventes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)

    # Afficher les statistiques pour vérification
    #print_statistiques(df)

    return df





df = processus_de_generation_complet(df, base_ventes=30, niveau_bruit=0.05)

In [ ]:
df= df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data.csv')


#  **Modélisation des Ventes de Snacks (Sucrés/Salés) : Méthodologie Technique**

*Ce notebook implémente un générateur de données synthétiques pour les ventes de snacks, avec une différenciation entre produits **sucrés** et **salés**. L'approche combine :*
 - Des **coefficients sectoriels** documentés
 - Une **modélisation météo-dépendante** fine
 - Des **effets saisonniers/culturels** (Ramadan, fêtes)
 - Un **bruit contrôlé** pour le réalisme

---

#**1. Coefficients Spécifiques par Type de Snack**

    special_day_coefficients = {
        'ramadan': {'sucré': 1.5, 'salé': 1.2},  # Forte demande en sucrés pendant le jeûne
     'new_year': {'sucré': 1.05, 'salé': 1.05} # Impact modéré uniforme
       }

# **Justification scientifique** :
 - **Ramadan** :
   - +50% sur les sucrés (besoin énergétique post-jeûne, études nutritionnelles
   - +20% sur les salés (rééquilibrage électrolytique)
 - **Weekends** :
   - +10% uniforme (consommation loisirs)

 ---

#  **2. Impact Météorologique Différencié**
**Modèle thermique** :
``` python
    if type_snack == 'sucré':
     if temp_jour < 15: temp_effect = 1.2  # Réconfort par temps froid
    else:  # Salé
     if temp_jour > 25: temp_effect = 1.2  # Compensation sudation
```

# **Effets précipitations** :
| Type       | Pluie Légère | Forte Pluie |
|------------|--------------|-------------|
| **Sucré**  | +15%         | +10%        |
| **Salé**   | -5%          | -25%        |


---
# **Contrôles Qualité** :
 - Plafonds de ventes (`max_ventes_jour_normal`)
 - Vérification des distributions via `print_statistiques_snacks()`

---



In [ ]:

# Impact des jours spéciaux
special_day_coefficients = {
    'Weekend': {'sucré': 1.1, 'salé': 1.1},
    'jour_ferie': {'sucré': 1.0, 'salé': 1.0},#pas dimpact
    'vacances_scolaires': {'sucré': 1.0, 'salé': 1.0}, #pas d'impact
    'ramadan': {'sucré': 1.5, 'salé': 1.2},  # Fort impact sur les sucrés après rupture
    'new_year': {'sucré': 1.05, 'salé': 1.05}
}

# Valeurs maximales quotidiennes
max_ventes_jour_normal = {
    'sucré': 30,
    'salé': 40
}

max_ventes_jour_special = {
    'sucré': 30,
    'salé': 40
}

def impact_meteo_snack(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_snack):

    # Effet température de base
    if type_snack == 'sucré':
        if temp_jour < 15:
            temp_effect = 1.2  # +20% en dessous de 15°C pour le sucré
        elif 15 <= temp_jour < 25:
            temp_effect = 1.0  # Neutre
        else:
            temp_effect = 0.9  # -10% au-dessus de 25°C pour le sucré
    else:  # Salé
        if temp_jour < 15:
            temp_effect = 0.9  # -10% en dessous de 15°C pour le salé
        elif 15 <= temp_jour < 25:
            temp_effect = 1.0  # Neutre
        else:
            temp_effect = 1.2  # +20% au-dessus de 25°C pour le salé

    # Effet précipitations
    if type_snack == 'sucré':
        # Les jours pluvieux incitent à la consommation de douceurs réconfortantes
        if precip_jour == 0:
            precip_effect = 1.0  # Pas de pluie, pas d'effet
        elif 0 < precip_jour <= 5:
            precip_effect = 1.15  # Pluie légère: +15% (effet réconfort)
        elif 5 < precip_jour <= 15:
            precip_effect = 1.25  # Pluie modérée: +25% (fort effet réconfort)
        else:
            precip_effect = 1.1  # Forte pluie: +10% (réconfort mais mobilité réduite)
    else:  # Salé
        # La pluie réduit les activités extérieures où le salé est populaire
        if precip_jour == 0:
            precip_effect = 1.0  # Pas de pluie, pas d'effet
        elif 0 < precip_jour <= 5:
            precip_effect = 0.95  # Pluie légère: -5%
        elif 5 < precip_jour <= 15:
            precip_effect = 0.85  # Pluie modérée: -15%
        else:
            precip_effect = 0.75  # Forte pluie: -25% (forte réduction des activités extérieures)

    # Effet humidité ajouté
    if type_snack == 'sucré':
        # L'humidité élevée augmente le besoin de sucre pour l'énergie
        if humidity_jour < 40:
            humidity_effect = 0.95  # Air sec: -5% (moins de fatigue)
        elif 40 <= humidity_jour < 70:
            humidity_effect = 1.0  # Humidité modérée: neutre
        else:
            humidity_effect = 1.12  # Humidité élevée: +12% (fatigue accrue, besoin d'énergie)
    else:  # Salé
        # L'humidité élevée augmente la transpiration et le besoin de sel
        if humidity_jour < 40:
            humidity_effect = 0.90  # Air sec: -10% (moins de perte de sel)
        elif 40 <= humidity_jour < 70:
            humidity_effect = 1.0  # Humidité modérée: neutre
        else:
            humidity_effect = 1.18  # Humidité élevée: +18% (besoin de reconstituer le sel perdu)

    # Effet vent
    if wind_speed_jour < 20:
        wind_effect = 1.0  # Vent faible à modéré: pas d'impact
    else:
        wind_effect = 0.95  # Vent fort: légère baisse des ventes (-5%) car inconfort extérieur

    # Combinaison des effets
    return max(0.5, temp_effect * precip_effect * humidity_effect * wind_effect)
def calcul_impact_climatique_snack(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_snack):

    impact = impact_meteo_snack(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_snack)
    return impact

def generer_ventes_snacks_base(df, base_ventes_sucre=20, base_ventes_sale=20):


    # Génération pour les snacks sucrés
    df['snacks_sucré'] = df.apply(
        lambda row: calcul_impact_climatique_snack(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'sucré'
        ) * base_ventes_sucre,
        axis=1
    )

    # Génération pour les snacks salés
    df['snacks_salé'] = df.apply(
        lambda row: calcul_impact_climatique_snack(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'salé'
        ) * base_ventes_sale,
        axis=1
    )

    return df

def ajuster_ventes_snacks(df):

    for type_snack in ['sucré', 'salé']:

        # Coefficients pour jours spéciaux
        df[f'special_coeff_{type_snack}'] = 1.0

        # Appliquer les coefficients de jour spécial
        for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
            if day_type in df.columns:
                mask = df[day_type] == 1
                if mask.any() and day_type in special_day_coefficients:
                    df.loc[mask, f'special_coeff_{type_snack}'] = special_day_coefficients[day_type][type_snack]


        if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
            both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
            if both_mask.any():
                for mask_row in df[both_mask].index:

                    max_coeff = max(
                        special_day_coefficients['jour_ferie'][type_snack],
                        special_day_coefficients['Weekend'][type_snack]
                    ) * 1.1
                    df.loc[mask_row, f'special_coeff_{type_snack}'] = max_coeff




        # Calcul final des ventes
        df[f'snacks_{type_snack}'] = (
            df[f'snacks_{type_snack}'] *

            df[f'special_coeff_{type_snack}']

        )

        # Limitation des ventes journalières
        # Jours normaux
        normal_mask = (~df['ramadan'].astype(bool)) & (~df['new_year'].astype(bool))
        df.loc[
            normal_mask & (df[f'snacks_{type_snack}'] > max_ventes_jour_normal[type_snack]),
            f'snacks_{type_snack}'
        ] = max_ventes_jour_normal[type_snack]

        # Jours spéciaux (Ramadan, New Year)
        special_mask = (df['ramadan'].astype(bool) | df['new_year'].astype(bool))
        df.loc[special_mask, f'snacks_{type_snack}'] = df.loc[
            special_mask, f'snacks_{type_snack}'
        ].clip(upper=max_ventes_jour_special[type_snack])

    # Nettoyer les colonnes temporaires utilisées pour les calculs
    cols_to_drop = [col for col in df.columns if  col.startswith('special_coeff_')]


    return df.drop(columns=cols_to_drop, errors='ignore')

def ajouter_bruit(df, niveau_bruit_mult=0.08):

    import numpy as np

    df_avec_bruit = df.copy()

    for type_snack in ['sucré', 'salé']:
        col = f'snacks_{type_snack}'



        # Bruit multiplicatif
        facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit_mult, size=len(df))
        df_avec_bruit[col] = df_avec_bruit[col] * facteurs_multiplicatifs


        df_avec_bruit[col] = np.maximum(1, df_avec_bruit[col])


        df_avec_bruit[col] = df_avec_bruit[col].round().astype(int)

    return df_avec_bruit

def print_statistiques_snacks(df):

    print("=== STATISTIQUES DES VENTES DE SNACKS ===")

    for type_snack in ['sucré', 'salé']:
        col = f'snacks_{type_snack}'
        print(f"\n--- SNACKS {type_snack.upper()} ---")
        print(f"Ventes moyennes globales: {df[col].mean():.2f}")


        if 'ramadan' in df.columns:
            print(f"Ventes pendant Ramadan: {df.loc[df['ramadan'] == 1, col].mean():.2f}")
            print(f"Ventes hors Ramadan: {df.loc[df['ramadan'] == 0, col].mean():.2f}")

        if 'Weekend' in df.columns:
            print(f"Ventes pendant Weekend: {df.loc[df['Weekend'] == 1, col].mean():.2f}")
            print(f"Ventes hors weekend: {df.loc[df['Weekend'] == 0, col].mean():.2f}")

        if 'new_year' in df.columns:
            print(f"Ventes pendant New Year: {df.loc[df['new_year'] == 1, col].mean():.2f}")
            print(f"Ventes hors New Year: {df.loc[df['new_year'] == 0, col].mean():.2f}")


        if 'profil_cotier' in df.columns:
            for region in df['profil_cotier'].unique():
                print(f"Ventes en région {region}: {df.loc[df['profil_cotier'] == region, col].mean():.2f}")

    print("\n===============================")

def processus_de_generation_complet_snacks(df, base_ventes_sucre=20, base_ventes_sale=20, niveau_bruit_add=0.05, niveau_bruit_mult=0.08):


    df = generer_ventes_snacks_base(df, base_ventes_sucre=base_ventes_sucre, base_ventes_sale=base_ventes_sale)


    df = ajuster_ventes_snacks(df)


    df = ajouter_bruit(df, niveau_bruit_mult=niveau_bruit_mult)


    print_statistiques_snacks(df)

    return df
df = processus_de_generation_complet_snacks(df, base_ventes_sucre=20, base_ventes_sale=20)

In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data.csv')
print(df.shape)

(43769, 49)



#  **Modélisation des Ventes de Produits Laitiers**

**Ce module génère des données synthétiques de ventes pour 4 catégories de produits laitiers :**
 - **Frais** (yaourts, fromages blancs)
 - **Fromages**
 - **Lait & boissons lactées**
 - **Glaces**

 ---

## **1. Modélisation Météo-Spécifique**
 **Impact différencié par température** :
 ```python
 if type_produit == 'glace':
     if temp_jour < 10: temp_effect = 0.5  # -50% par temps froid
     elif temp_jour > 30: temp_effect = 1.6  # +60% en canicule
 ```

**Comportements observés** :

 | Catégorie          | Température Optimale | Effet Maximal |
 |--------------------|----------------------|---------------|
 | Produits frais     | >25°C                | +30%          |
 | Fromages           | <10°C                | +10%          |
 | Glaces             | >30°C                | +60%          |
 | Lait               | <10°C                | +15%          |

 ---

 ## **2. Effets Culturels (Ramadan)**
 ```python
 special_day_coefficients_laitiers = {
     'ramadan': 1.5,  # +50% global
     'new_year': 1.0   # neutre
 }
 ```

# **Ajustements fins** :
 - **Lait** : Bonus supplémentaire (+30%) pour rupture du jeûne
 - **Glaces** : +20% comme dessert festif


 ---



## **Mécanismes clés** :
 - Effets multiplicatifs entre variables
 - Plafonnement réaliste (`max_ventes_jour_normal_laitiers`)
 - Bruit gaussien (σ=8%)

---

## **4. Validation Statistique**
 ```python
 print_statistiques_produits_laitiers(df)
 ```

In [ ]:
# Impact des jours spéciaux sur les produits laitiers
special_day_coefficients_laitiers = {
    'Weekend': 1.0,#neutre
    'jour_ferie': 1.0,
    'vacances_scolaires': 1.0,
    'ramadan': 1.5,  # +50% pendant le Ramadan
    'new_year': 1.0
}

# Valeurs maximales quotidiennes
max_ventes_jour_normal_laitiers = 150
max_ventes_jour_special_laitiers = 150

# Coefficients par type de produit laitier
types_produits_laitiers = ['frais', 'fromage', 'lait,boissons lactées', 'glace']

def meteo_produits_laitiers_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_produit):


    # Effet température selon le type de produit
    if type_produit == 'frais':  # yaourts, fromage blanc frais
        if temp_jour < 10:
            temp_effect = 0.85
        elif 10 <= temp_jour < 20:
            temp_effect = 1.05
        elif 20 <= temp_jour < 30:
            temp_effect = 1.2
        else:
            temp_effect = 1.3

    elif type_produit == 'glace':  # crèmes glacées
        if temp_jour < 10:
            temp_effect = 0.5
        elif 10 <= temp_jour < 20:
            temp_effect = 0.8
        elif 20 <= temp_jour < 30:
            temp_effect = 1.4
        else:
            temp_effect = 1.6

    elif type_produit == 'fromage':
        if temp_jour < 10:
            temp_effect = 1.1  # fromages à fondre en hiver
        elif 10 <= temp_jour < 20:
            temp_effect = 1.0
        else:
            temp_effect = 0.9  # baisse légère par temps chaud

    else:  # lait, boissons lactées
        if temp_jour < 10:
            temp_effect = 1.15  # boissons chaudes
        elif 10 <= temp_jour < 20:
            temp_effect = 1.1
        elif 20 <= temp_jour < 30:
            temp_effect = 0.95
        else:
            temp_effect = 0.9

    # Effet humidité
    if humidity_jour < 40:
        if type_produit == 'frais' or type_produit == 'lait,boissons lactées':
            humidity_effect = 1.05  # plus hydratant(+5%)
        else:
            humidity_effect = 1.0
    elif 40 <= humidity_jour < 70:
        humidity_effect = 1.0
    else:
        if type_produit == 'fromage':
            humidity_effect = 0.95  # moins appétissant
        elif type_produit == 'frais':
            humidity_effect = 1.08  # plus rafraîchissant
        elif type_produit == 'glace':
            humidity_effect = 1.12  # plus recherché quand il fait humide et chaud
        else:
            humidity_effect = 1.0

    # Effet précipitations
    if precip_jour == 0:
        if type_produit == 'glace':
            precip_effect = 1.0  # déjà il y a l'effet température
        else:
            precip_effect = 1.0
    elif 0 < precip_jour <= 5:
        precip_effect = 1.0  # effet négligeable
    else:
        if type_produit == 'glace':
            precip_effect = 0.8  # forte baisse des achats impulsifs
        elif type_produit == 'fromage' or type_produit == 'lait,boissons lactées':
            precip_effect = 1.05  # légère hausse (stockage, réconfort)
        else:
            precip_effect = 1.0

    # Effet vent
    if wind_speed_jour < 20:
        wind_effect = 1.0  # pas d'impact
    else:
        if type_produit == 'glace':
            wind_effect = 0.95  # légère baisse car inconfort extérieur
        else:
            wind_effect = 1.0  # pas d'impact significatif

    # Combinaison des effets
    return max(0.5, temp_effect * humidity_effect * precip_effect * wind_effect)

def calcul_impact_climatique_laitiers(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_produit):

    impact = meteo_produits_laitiers_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_produit)
    return impact

def generer_ventes_produits_laitiers_base(df, base_ventes_frais=60, base_ventes_fromage=60, base_ventes_lait=70, base_ventes_glace=50):


    # Génération pour les produits laitiers frais
    df['produits_laitiers_frais'] = df.apply(
        lambda row: calcul_impact_climatique_laitiers(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'frais'
        ) * base_ventes_frais,
        axis=1
    )

    # Génération pour les fromages
    df['produits_laitiers_fromage'] = df.apply(
        lambda row: calcul_impact_climatique_laitiers(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'fromage'
        ) * base_ventes_fromage,
        axis=1
    )

    # Génération pour le lait
    df['produits_laitiers_lait,boissons lactées'] = df.apply(
        lambda row: calcul_impact_climatique_laitiers(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'lait,boissons lactées'
        ) * base_ventes_lait,
        axis=1
    )

    # Génération pour les glaces
    df['produits_laitiers_glace'] = df.apply(
        lambda row: calcul_impact_climatique_laitiers(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'glace'
        ) * base_ventes_glace,
        axis=1
    )

    return df

def ajuster_ventes_produits_laitiers(df):


    # Coefficients pour jours spéciaux
    df['special_coeff_laitiers'] = 1.0

    # Appliquer les coefficients de jour spécial
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
        if day_type in df.columns:
            mask = df[day_type] == 1
            if mask.any() and day_type in special_day_coefficients_laitiers:
                df.loc[mask, 'special_coeff_laitiers'] = special_day_coefficients_laitiers[day_type]


    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():

            max_coeff = max(
                special_day_coefficients_laitiers['jour_ferie'],
                special_day_coefficients_laitiers['Weekend']
            ) * 1.1
            df.loc[both_mask, 'special_coeff_laitiers'] = max_coeff



    df['trend_effect'] = 1.0
    if 'temp_jour_lag1' in df.columns:
        delta_temp = df['temp_jour'] - df['temp_jour_lag1']
        # Augmentation des ventes de glaces et produits frais quand la température augmente soudainement
        df.loc[(delta_temp > 5) & (df['temp_jour'] > 25), 'trend_effect'] = 1.2

    # Appliquer les coefficients à chaque type de produit laitier
    for type_produit in types_produits_laitiers:
        col = f'produits_laitiers_{type_produit}'

        # Facteur d'ajustement spécifique pour le Ramadan selon le type de produit
        ramadan_factor = 1.0
        if 'ramadan' in df.columns:
            if type_produit == 'lait':
                ramadan_factor = 1.3  # Augmentation pour le lait pendant Ramadan
            elif type_produit == 'glace':
                ramadan_factor = 1.2  # Augmentation pour les desserts sucrés

        # Calcul final des ventes
        df[col] = (
            df[col] *
            df['special_coeff_laitiers'] *

            df['trend_effect']
        )

        # Ajustement spécifique pour Ramadan
        if 'ramadan' in df.columns:
            df.loc[df['ramadan'] == 1, col] *= ramadan_factor

        # Limitation des ventes journalières
        df[col] = df[col].clip(upper=max_ventes_jour_normal_laitiers)

        # Limitation spéciale pour les jours de fête
        special_mask = (df['ramadan'].astype(bool) | df['new_year'].astype(bool))
        df.loc[special_mask, col] = df.loc[special_mask, col].clip(upper=max_ventes_jour_special_laitiers)


    cols_to_drop = ['special_coeff_laitiers', 'trend_effect']
    return df.drop(columns=cols_to_drop, errors='ignore')

def ajouter_bruit_produits_laitiers(df, niveau_bruit=0.08):

    import numpy as np

    df_avec_bruit = df.copy()

    for type_produit in types_produits_laitiers:
        col = f'produits_laitiers_{type_produit}'


        facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))


        df_avec_bruit[col] = df_avec_bruit[col] * facteurs_multiplicatifs


        df_avec_bruit[col] = np.maximum(1, df_avec_bruit[col]).round().astype(int)

    return df_avec_bruit

def print_statistiques_produits_laitiers(df):

    print("=== STATISTIQUES DES VENTES DE PRODUITS LAITIERS ===")

    for type_produit in types_produits_laitiers:
        col = f'produits_laitiers_{type_produit}'
        print(f"\n--- PRODUITS LAITIERS {type_produit.upper()} ---")
        print(f"Ventes moyennes globales: {df[col].mean():.2f}")


        if 'ramadan' in df.columns:
            print(f"Ventes pendant Ramadan: {df.loc[df['ramadan'] == 1, col].mean():.2f}")
            print(f"Ventes hors Ramadan: {df.loc[df['ramadan'] == 0, col].mean():.2f}")

        if 'Weekend' in df.columns:
            print(f"Ventes pendant Weekend: {df.loc[df['Weekend'] == 1, col].mean():.2f}")
            print(f"Ventes hors weekend: {df.loc[df['Weekend'] == 0, col].mean():.2f}")

        if 'new_year' in df.columns:
            print(f"Ventes pendant New Year: {df.loc[df['new_year'] == 1, col].mean():.2f}")
            print(f"Ventes hors New Year: {df.loc[df['new_year'] == 0, col].mean():.2f}")


        if 'profil_cotier' in df.columns:
            for region in df['profil_cotier'].unique():
                print(f"Ventes en région {region}: {df.loc[df['profil_cotier'] == region, col].mean():.2f}")

    print("\n===============================")

def processus_de_generation_complet_produits_laitiers(df, base_ventes_frais=60, base_ventes_fromage=60,
                                                     base_ventes_lait=70, base_ventes_glace=50, niveau_bruit=0.08):
    """Applique tout le processus de génération de ventes de produits laitiers en une seule fonction."""


    df = generer_ventes_produits_laitiers_base(
        df,
        base_ventes_frais=base_ventes_frais,
        base_ventes_fromage=base_ventes_fromage,
        base_ventes_lait=base_ventes_lait,
        base_ventes_glace=base_ventes_glace
    )

    df = ajuster_ventes_produits_laitiers(df)


    df = ajouter_bruit_produits_laitiers(df, niveau_bruit=niveau_bruit)


    print_statistiques_produits_laitiers(df)

    return df


df = processus_de_generation_complet_produits_laitiers(df)

=== STATISTIQUES DES VENTES DE PRODUITS LAITIERS ===

--- PRODUITS LAITIERS FRAIS ---
Ventes moyennes globales: 72.27
Ventes pendant Ramadan: 100.59
Ventes hors Ramadan: 69.70
Ventes pendant Weekend: 72.43
Ventes hors weekend: 72.20
Ventes pendant New Year: 63.48
Ventes hors New Year: 72.29
Ventes en région Hors côtier: 72.23
Ventes en région Intérieure: 72.29
Ventes en région Côtière: 72.26

--- PRODUITS LAITIERS FROMAGE ---
Ventes moyennes globales: 59.45
Ventes pendant Ramadan: 86.29
Ventes hors Ramadan: 57.02
Ventes pendant Weekend: 59.51
Ventes hors weekend: 59.43
Ventes pendant New Year: 60.68
Ventes hors New Year: 59.45
Ventes en région Hors côtier: 59.56
Ventes en région Intérieure: 59.45
Ventes en région Côtière: 59.32

--- PRODUITS LAITIERS LAIT,BOISSONS LACTÉES ---
Ventes moyennes globales: 74.97
Ventes pendant Ramadan: 109.54
Ventes hors Ramadan: 71.84
Ventes pendant Weekend: 75.06
Ventes hors weekend: 74.94
Ventes pendant New Year: 78.41
Ventes hors New Year: 74.97
Ventes 

In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
import pandas as pd
df=pd.read_csv('data1.csv')
df.rename(columns={
    'produits_laitiers_fromages': 'produits_laitiers(fromages)',
    'produits_laitiers_lait,boissons lactées': 'produits_laitiers(lait,boissons lactées)',
    'produits_laitiers_glace': 'produits_laitiers(lait glacé,crème glacée)',
    'produits_laitiers_frais': 'produits_laitiers_frais(yaourts, fromage blanc)'
}, inplace=True)

In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data.csv')

In [ ]:
import numpy as np


#  **Modélisation des Ventes de Boissons Chaudes**

 Ce module implémente un générateur de données synthétiques pour les boissons chaudes (café, thé, chocolat chaud) avec une logique **inverse** des boissons fraîches : la demande augmente quand la température baisse.

 ---

## **1. Effets Climatiques Inversés**
 **Courbe d'impact thermique** :
 ```python
 if temp_jour < 5: temp_effect = 1.5  # +50%
 elif 5-15°C: temp_effect = 1.5 - 0.03*(temp-5)  # Décroissance linéaire
 elif >25°C: temp_effect = max(0.5, 0.8 - 0.02*(temp-25))  # Minimum à 50%
 ```

 **Autres facteurs** :
 - **Précipitations** : +6% par mm de pluie (plafonné à +60%)
 - **Vent** : +1% par km/h (amplifie la sensation de froid)
 - **Humidité** : +0.5% par point >50% d'humidité quand <15°C


 ---

## **2. Particularités Culturelles (Ramadan)**
 ```python
 special_day_coefficients_chauds = {
     'ramadan': 1.9,  # +90% (consommation nocturne)
     'new_year': 1.0   # neutre (contrairement aux boissons fraîches)
 }
 ```

### **Mécanisme** :  
 Pendant le Ramadan, les boissons chaudes sont consommées :
 - Avant l'aube (Suhoor)
 - Pour rompre le jeûne (Iftar)
 - Avec des effets stimulants (café/thé)

 ---




 ---




In [ ]:
def meteo_boisson_chaude_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    if temp_jour < 5:
        temp_effect = 1.7  # Augmenter de 1.5 à 1.7 (+70% des ventes normales)
    elif 5 <= temp_jour < 15:
        temp_effect = 1.7 - 0.04 * (temp_jour - 5)  # -4% par °C (augmenté de 3% à 4%)
    elif 15 <= temp_jour < 25:
        temp_effect = 1.3 - 0.05 * (temp_jour - 15)  # -5% par °C (augmenté de 4% à 5%)
    else:
        temp_effect = 0.8 - 0.03 * (temp_jour - 25)  # Diminution plus accentuée
        temp_effect = max(temp_effect, 0.4)  # Minimum abaissé à 40%

    # Effet précipitations (augmente les ventes pour les boissons chaudes)
    precip_effect = min(1.6, 1 + (precip_jour * 0.06))  # +6% par mm, plafonné à +60%

    # Effet vent (augmente les ventes car sensation de froid)
    wind_effect = 1.0
    if temp_jour < 20:  # Le vent a plus d'impact quand il fait déjà frais
        wind_effect = 1 + (wind_speed_jour / 100)  # +1% par km/h de vent

    # Effet humidité (augmente la sensation de froid quand il fait frais)
    if temp_jour < 15:
        # Augmentation de 0.5% pour chaque point d'humidité au-dessus de 50%
        humidity_effect = 1 + 0.005 * max(0, humidity_jour - 50)
    else:
        # Réduction quand il fait chaud et humide
        humidity_effect = 1 - 0.002 * max(0, humidity_jour - 50)

    # Combinaison des effets
    return min(2.0, max(0.5, temp_effect * precip_effect * wind_effect * humidity_effect))  # Entre 50% et 200% des ventes de base

def calcul_impact_climatique_chaud(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    """Calcul de l'impact climatique pour les boissons chaudes"""
    impact = meteo_boisson_chaude_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)
    return impact

def generer_ventes_boissons_chaudes(df, base_ventes=20):

    df['boissons_chaudes'] = df.apply(
        lambda row: calcul_impact_climatique_chaud(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

def ajuster_ventes_chaudes(df):

    # Coefficients pour jours spéciaux - Spécifiques aux boissons chaudes
    special_day_coefficients_chauds = {
    'Weekend': 1.0,
    'jour_ferie': 1.0,
    'vacances_scolaires': 1.0,
    'ramadan': 1.5,  #  1.5 (+50% au lieu de +90%)
    'new_year': 1.2  # Légère augmentation pour le nouvel an
}

    df['special_coeff_chaud'] = 1.0

    # Appliquer les coefficients de jour spécial
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
        if day_type in df.columns:
            mask = df[day_type] == 1
            if mask.any():
                df.loc[mask, 'special_coeff_chaud'] = special_day_coefficients_chauds[day_type]


    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            df.loc[both_mask, 'special_coeff_chaud'] = max(
                special_day_coefficients_chauds['jour_ferie'],
                special_day_coefficients_chauds['Weekend']
            ) * 1.15  # Bonus de 15% pour combinaison


    df['trend_effect_chaud'] = 1.0
    if 'temp_jour_lag1' in df.columns:
        delta_temp = df['temp_jour_lag1'] - df['temp_jour']  # Augmentation quand température baisse
        df.loc[(delta_temp > 5) & (df['temp_jour'] < 15), 'trend_effect_chaud'] = 1.4  # +40% si chute soudaine


    densite_map_chaud = {
        'Grande métropole': 1.7,    # +70% (vie professionnelle active)
        'Ville moyenne': 1.4,       # +40%
        'Petite ville': 1.2         # +20%
    }
    df['densite_effect_chaud'] = df['densite_urbaine'].map(densite_map_chaud)



    # Calcul final des ventes
    df['boissons_chaudes'] = df['boissons_chaudes'] * df['special_coeff_chaud'] * df['trend_effect_chaud'] * df['densite_effect_chaud']

    # Limites de ventes maximales journalières
    max_ventes_jour_normal_chaud = 20   # Différent des boissons fraîches
    max_ventes_jour_special_chaud = 30


    df.loc[
        (~df['ramadan'].astype(bool)) & (~df['new_year'].astype(bool)) & (df['boissons_chaudes'] > max_ventes_jour_normal_chaud),
        'boissons_chaudes'
    ] = max_ventes_jour_normal_chaud


    df.loc[
        (df['ramadan'].astype(bool) | df['new_year'].astype(bool)),
        'boissons_chaudes'
    ] = df.loc[
        (df['ramadan'].astype(bool) | df['new_year'].astype(bool)),
        'boissons_chaudes'
    ].clip(upper=max_ventes_jour_special_chaud)

    # Effet d'interaction avec les boissons fraîches
    if 'boissons_fraiches' in df.columns:
        # quand les boissons fraîches sont très hautes, impact négatif sur les chaudes
        df.loc[df['boissons_fraiches'] > 37, 'boissons_chaudes'] *= 0.9

    return df


def ajouter_bruit_chaud(df, niveau_bruit=0.05):

    df_avec_bruit = df.copy()


    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))


    df_avec_bruit["boissons_chaudes"] = df["boissons_chaudes"] * facteurs_multiplicatifs

    df_avec_bruit["boissons_chaudes"] = np.maximum(1, df_avec_bruit["boissons_chaudes"])


    df_avec_bruit["boissons_chaudes"] = df_avec_bruit["boissons_chaudes"].round().astype(int)

    return df_avec_bruit

def processus_generation_boissons_chaudes(df, base_ventes=10, niveau_bruit=0.05):



    df = generer_ventes_boissons_chaudes(df, base_ventes=base_ventes)


    df = ajuster_ventes_chaudes(df)


    df = ajouter_bruit_chaud(df, niveau_bruit=niveau_bruit)


    print_statistiques_chaudes(df)

    return df

def print_statistiques_chaudes(df):

    print("=== STATISTIQUES DES VENTES DE BOISSONS CHAUDES ===")
    print(f"Ventes moyennes globales: {df['boissons_chaudes'].mean():.2f}")


    if 'ramadan' in df.columns:
        print(f"Ventes moyennes pendant Ramadan: {df.loc[df['ramadan'] == 1, 'boissons_chaudes'].mean():.2f}")
        print(f"Ventes moyennes hors Ramadan: {df.loc[df['ramadan'] == 0, 'boissons_chaudes'].mean():.2f}")





    print("\nImpact de la température:")
    temp_ranges = [(-10, 0), (0, 10), (10, 20), (20, 30), (30, 50)]
    for temp_min, temp_max in temp_ranges:
        mask = (df['temp_jour'] >= temp_min) & (df['temp_jour'] < temp_max)
        if mask.any():
            print(f"Température {temp_min}-{temp_max}°C: {df.loc[mask, 'boissons_chaudes'].mean():.2f}")

    print("==============================================")
df = processus_generation_boissons_chaudes(df, base_ventes=10, niveau_bruit=0.05)


#  **Modélisation des Ventes de Produits Anti-Moustiques : Approche Entomoclimatique**

 **Ce module génère des données synthétiques de ventes de produits anti-moustiques en se basant sur des facteurs climatiques et environnementaux favorables au développement des moustiques.**

 ---

## **1. Modélisation des Conditions Optimales pour les Moustiques**
 ### **Courbe d'impact thermique** :
 ```python
 if temp < 15: effect = 0.3  # Très faible activité
 elif 25-35°C: effect = 1.0 + 0.08*(temp-25)  # Zone optimale
 elif >35°C: effect = 1.8 - 0.06*(temp-35)  # Décroissance
 ```

### **Autres facteurs clés** :
 - **Humidité** : +0.015% par point >40% (max à +100%)
 - **Précipitations** : Effet cumulatif sur 3 jours (+10% par 10mm)
 - **Vent** : -2% par km/h (dissipe les moustiques)



 ---

## **2. Saisonnalité Mensuelle**
 ```python
 monthly_coeff = {
     7: 2.8,  # Pic estival (juillet)
     1: 0.2   # Minimum hivernal (janvier)
 }
 ```

### **Justification biologique** :
 - Cycle de reproduction des moustiques


 ---

## **3. Effets Environnementaux**
 **Impact urbain** :

 | Type de Zone       | Coefficient | Explication |
 |--------------------|-------------|-------------|
 | Grande métropole   | 1.3         | Eaux stagnantes |
 | Ville moyenne      | 1.2         | |
 | Petite ville       | 1.0         | Base |

### **Effets climatiques régionaux** :
 - **Méditerranéen** : +20% (climat favorable)
 - **Désertique** : -20% (conditions hostiles)

 ---


In [ ]:

# Coefficients mensuels reflétant la saisonnalité des moustiques
monthly_coeff = {
    1: 0.2, 2: 0.4, 3: 0.6,   # Hiver → Faible
    4: 1.2, 5: 1.8, 6: 2.5,   # Printemps → Augmentation
    7: 2.8, 8: 2.6,           # Été → Pic
    9: 1.8, 10: 1.4, 11: 0.8, 12: 0.4  # Automne/Hiver → Diminution
}



def impact_meteo_anti_moustiques(temp, humidity, precip_j3, wind_speed):


    score = 0

    # Effet température - Optimale entre 25-35°C pour les moustiques
    if temp < 15:
        temp_effect = 0.3  # Très peu de moustiques
    elif 15 <= temp < 25:
        temp_effect = 0.3 + 0.07 * (temp - 15)  # Augmentation progressive
    elif 25 <= temp < 35:
        temp_effect = 1.0 + 0.08 * (temp - 25)  # Conditions optimales
    else:  # temp >= 35
        temp_effect = 1.8 - 0.06 * (temp - 35)  # Diminution quand trop chaud

    # Effet humidité - Les moustiques prospèrent dans l'humidité
    if humidity < 40:
        humidity_effect = 0.7  # Conditions défavorables
    elif 40 <= humidity < 60:
        humidity_effect = 0.7 + 0.015 * (humidity - 40)  # Augmentation progressive
    else:  # humidity >= 60
        humidity_effect = 1.0 + 0.02 * (humidity - 60)  # Conditions favorables

    # Effet précipitations des derniers jours (eau stagnante = reproduction)
    # Plus important après quelques jours de pluie (permet l'éclosion des œufs)
    if precip_j3 < 5:
        precip_effect = 1.0 + 0.05 * precip_j3  # Légère augmentation
    else:
        precip_effect = 1.25 + 0.1 * min((precip_j3 - 5) / 10, 3)  # +10% par 10mm, plafonné à +30%

    # Effet vent - Les moustiques sont moins actifs quand il y a du vent
    wind_effect = max(0.5, 1.0 - 0.02 * wind_speed)  # -2% par km/h, minimum 50%

    # Combiner les effets
    combined_effect = temp_effect * humidity_effect * precip_effect * wind_effect
    return combined_effect

def generer_ventes_anti_moustiques(df, base_ventes=50, niveau_bruit=0.1):


    df_result = df.copy()


    df_result['impact_meteo_moustiques'] = df_result.apply(
        lambda row: impact_meteo_anti_moustiques(
            row['temp_jour'],
            row['humidity_jour'],
            row.get('precip_j3', row['precip_jour']),
            row['wind_speed_jour']
        ),
        axis=1
    )



    df_result['trend_effect_moustiques'] = 1.0

    if all(col in df_result.columns for col in ['temp_jour_lag1', 'temp_jour_lag2', 'humidity_jour_lag1']):
        # Si 3 jours consécutifs chauds et humides, forte augmentation
        hot_humid_days = (
            (df_result['temp_jour'] > 25) &
            (df_result['temp_jour_lag1'] > 25) &
            (df_result['temp_jour_lag2'] > 25) &
            (df_result['humidity_jour'] > 60) &
            (df_result['humidity_jour_lag1'] > 60)
        )
        df_result.loc[hot_humid_days, 'trend_effect_moustiques'] = 1.8  # +80% après plusieurs jours favorables

    # . Effet urbanisation (plus d'eau stagnante dans certaines zones)
    if 'densite_urbaine' in df_result.columns:
        densite_map_moustiques = {
            'Grande métropole': 1.3,  # +30% (eau stagnante dans les zones urbaines)
            'Ville moyenne': 1.2,     # +20%
            'Petite ville': 1.0       # Base
        }
        df_result['densite_effect_moustiques'] = df_result['densite_urbaine'].map(densite_map_moustiques)
    else:
        df_result['densite_effect_moustiques'] = 1.0

    # 8. Effet région climatique
    if 'region_climatique' in df_result.columns:
        climat_map_moustiques = {
            'Méditerranéenne': 1.2,      # +20% (climat favorable)
            'Semi-aride': 0.9,           # -10% (moins favorable)
            'Aride/désertique': 0.8     # -20% (défavorable)
        }
        df_result['climat_effect_moustiques'] = df_result['region_climatique'].map(climat_map_moustiques)
    else:
        df_result['climat_effect_moustiques'] = 1.0


    # 10. Calcul des ventes
    df_result['ventes_anti_moustiques'] = (
        base_ventes *


        df_result['impact_meteo_moustiques'] *

        df_result['trend_effect_moustiques'] *
        df_result['densite_effect_moustiques'] *
        df_result['climat_effect_moustiques']

    )

    df_result['ventes_anti_moustiques'] = df_result['ventes_anti_moustiques'].clip(upper=20)

    return df_result

def ajouter_bruit(df, niveau_bruit=0.1):
        df_avec_bruit = df.copy()
        facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))
        df_avec_bruit["ventes_anti_moustiques"] = df["ventes_anti_moustiques"] * facteurs_multiplicatifs
        df_avec_bruit["ventes_anti_moustiques"] = np.maximum(0, df_avec_bruit["ventes_anti_moustiques"])
        df_avec_bruit["ventes_anti_moustiques"] = df_avec_bruit["ventes_anti_moustiques"].round().astype(int)
        return df_avec_bruit

def print_statistiques_anti_moustiques(df):
    if 'ventes_anti_moustiques' not in df.columns:
        print("La colonne 'ventes_anti_moustiques' n'existe pas dans le DataFrame.")
        return

    print("=== STATISTIQUES DES VENTES DE PRODUITS ANTI-MOUSTIQUES ===")
    print(f"Ventes moyennes globales: {df['ventes_anti_moustiques'].mean():.2f}")


    print("\nImpact de la température:")
    temp_ranges = [(0, 15), (15, 25), (25, 35), (35, 50)]
    for temp_min, temp_max in temp_ranges:
        mask = (df['temp_jour'] >= temp_min) & (df['temp_jour'] < temp_max)
        if mask.any():
            print(f"Température {temp_min}-{temp_max}°C: {df.loc[mask, 'ventes_anti_moustiques'].mean():.2f}")


    print("\nImpact de l'humidité:")
    humidity_ranges = [(0, 40), (40, 60), (60, 80), (80, 100)]
    for hum_min, hum_max in humidity_ranges:
        mask = (df['humidity_jour'] >= hum_min) & (df['humidity_jour'] < hum_max)
        if mask.any():
            print(f"Humidité {hum_min}-{hum_max}%: {df.loc[mask, 'ventes_anti_moustiques'].mean():.2f}")

    print("==============================================")

def processus_complet_anti_moustiques(df, base_ventes=10, niveau_bruit=0.1):
    df = generer_ventes_anti_moustiques(df, base_ventes, niveau_bruit)
    df = ajouter_bruit(df, niveau_bruit)
    print_statistiques_anti_moustiques(df)
    return df
df= processus_complet_anti_moustiques(df, base_ventes=10, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE PRODUITS ANTI-MOUSTIQUES ===
Ventes moyennes globales: 33.80

Impact de la température:
Température 0-15°C: 17.41
Température 15-25°C: 29.44
Température 25-35°C: 50.87
Température 35-50°C: 54.22

Impact de l'humidité:
Humidité 0-40%: 46.77
Humidité 40-60%: 36.95
Humidité 60-80%: 29.49
Humidité 80-100%: 27.93


In [ ]:
df.rename(columns={'ventes_anti_moustiques':'produits anti_moustiques'},inplace=True)

In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
import pandas as pd
df=pd.read_csv('data.csv')
print(df.shape)

(43769, 49)



#  **Modélisation des Ventes de Produits de Jardinage : Approche Saisonnière et Climatique**

 **Ce module génère des données synthétiques réalistes pour les ventes de produits de jardinage en intégrant les facteurs saisonniers, météorologiques et comportementaux.**

 ---

## **1. Saisonnalité Mensuelle des Activités de Jardinage**
 **Coefficients mensuels** :
 ```python
 mois_jardinage_coefficients = {
     3: 1.5,  # Mars - Début printemps (+50%)
     4: 1.8,  # Avril - Pic printanier (+80%)
     9: 1.6   # Septembre - Redémarrage automnal (+60%)
 }
 ```

## **Justifications** :
 - **Printemps** : Période de plantation et préparation des jardins
 - **Automne** : Plantation des bulbes et préparation hivernale
 ---

## **2. Impact des Conditions Météorologiques**
 **Courbes d'effet** :

 | Paramètre      | Effet Optimal               | Impact Maximum |
 |----------------|-----------------------------|----------------|
 | Température    | 15-25°C                     | +20% à 20°C    |
 | Précipitations | 5-15mm (pluie modérée)      | Neutre         |
 | Vent           | <10 km/h                    | Neutre         |
 | Humidité       | 40-70%                      | Optimal        |

### **Modélisation** :
 ```python
 if 15 <= temp_jour < 25:
     temp_effect = 0.8 + 0.02*(temp-15)  # Croissance linéaire
 elif temp_jour >= 30:
     temp_effect = 0.85 - 0.05*(temp-30)  # Forte décroissance
 ```

 ---

## **3. Comportements d'Achat**
### **Jours favorables** :
 - **Weekends** : Coefficient 1.8 (+80%) - temps libre
 - **Jours fériés** : 1.4 (+40%) - opportunité de jardinage
 - **Après sécheresse** : +30% quand la pluie arrive

### **Effet urbain** :
 | Zone Urbaine       | Coefficient | Explication               |
 |--------------------|-------------|---------------------------|
 | Petite ville       | 1.3         | Maisons avec jardins       |
 | Grande métropole   | 0.7         | Moins d'espaces verts      |

 ---



In [ ]:
import numpy as np

def meteo_jardinage_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    # Effet température
    if temp_jour < 5:
        temp_effect = 0.3
    elif 5 <= temp_jour < 15:
        temp_effect = 0.3 + 0.05 * (temp_jour - 5)
    elif 15 <= temp_jour < 25:
        temp_effect = 0.8 + 0.02 * (temp_jour - 15)
    elif 25 <= temp_jour < 30:
        temp_effect = 1.0 - 0.03 * (temp_jour - 25)
    else:
        temp_effect = max(0.2, 0.85 - 0.05 * (temp_jour - 30))

    # Effet précipitations
    if precip_jour == 0:
        precip_effect = 0.8
    elif 0 < precip_jour <= 5:
        precip_effect = 0.8 + 0.04 * precip_jour
    elif 5 < precip_jour <= 15:
        precip_effect = 1.0
    else:
        precip_effect = max(0.4, 1.0 - 0.04 * (precip_jour - 15))

    # Effet vent
    if wind_speed_jour < 10:
        wind_effect = 1.0
    elif 10 <= wind_speed_jour < 20:
        wind_effect = 1.0 - 0.02 * (wind_speed_jour - 10)
    else:
        wind_effect = max(0.5, 0.8 - 0.03 * (wind_speed_jour - 20))

    # Effet humidité
    if 40 <= humidity_jour <= 70:
        humidity_effect = 1.0
    elif humidity_jour < 40:
        humidity_effect = 0.9 - 0.01 * (40 - humidity_jour)
    else:
        humidity_effect = 0.9 - 0.01 * (humidity_jour - 70)

    return max(0.2, temp_effect * precip_effect * wind_effect * humidity_effect)

def calcul_impact_climatique_jardinage(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    return meteo_jardinage_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)

def generer_ventes_jardinage_base(df, base_ventes=5):
    # Calcul de l'effet météo indépendamment du profil côtier
    df['impact_climatique'] = df.apply(
        lambda row: calcul_impact_climatique_jardinage(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ),
        axis=1
    )

    # Étape 1 : Générer les ventes pour les hors côtières
    df['produits_jardinage'] = df['impact_climatique'] * base_ventes

    # Étape 2 : Appliquer des multiplicateurs selon la règle
    df.loc[df['profil_cotier'] == 'Côtière', 'produits_jardinage'] *= 1.2  # +10%
    df.loc[df['profil_cotier'] == 'Intérieure', 'produits_jardinage'] *= 1.5  # 1.10 * 1.20 = +32%

    # Pas de ventes négatives
    df['produits_jardinage'] = df['produits_jardinage'].clip(lower=0)

    return df



# Coefficients jours spéciaux
special_day_jardinage_coefficients = {
    'jour_ferie': 1.4,
    'Weekend': 1.8,
    'vacances_scolaires': 1.2,
    'jour_pluie_apres_secheresse': 1.5,  # j'ai ajouté ce coefficient car tu l'utilises dans ton code
}

max_ventes_jour_normal = 15
max_ventes_jour_special = 20

def ajuster_ventes_jardinage(df):
    
    # Coefficients jours spéciaux
    df['special_coeff'] = 1.0
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires']:
        if day_type in df.columns:
            mask = df[day_type] == 1
            if day_type in special_day_jardinage_coefficients and mask.any():
                df.loc[mask, 'special_coeff'] = special_day_jardinage_coefficients[day_type]

    # Combinaison jour férié + week-end
    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            df.loc[both_mask, 'special_coeff'] = max(
                special_day_jardinage_coefficients['jour_ferie'],
                special_day_jardinage_coefficients['Weekend']
            ) * 1.1

    # Trend après sécheresse
    df['trend_effect'] = 1.0
    if all(col in df.columns for col in ['precip_jour_lag1', 'precip_jour_lag2', 'precip_jour_lag3', 'precip_jour']):
        secheresse_suivie_pluie = (
            (df['precip_jour_lag1'] < 2) &
            (df['precip_jour_lag2'] < 2) &
            (df['precip_jour_lag3'] < 2) &
            (df['precip_jour'] >= 5)
        )
        df.loc[secheresse_suivie_pluie, 'trend_effect'] = special_day_jardinage_coefficients['jour_pluie_apres_secheresse']

        df.loc[mask & ((df['Weekend'] == 1) | (df['jour_ferie'] == 1)), 'produits_jardinage'] = \
            df.loc[mask & ((df['Weekend'] == 1) | (df['jour_ferie'] == 1)), 'produits_jardinage'].clip(upper=20)

    
    # Appliquer tous les effets
    df['produits_jardinage'] = (
        df['produits_jardinage'] *
      
        df['special_coeff'] *
        df['trend_effect'] 
       
    )

    # Limitation ventes par jour
    df.loc[
        (df['Weekend'] == 0) & (df['jour_ferie'] == 0),
        'produits_jardinage'
    ] = df.loc[
        (df['Weekend'] == 0) & (df['jour_ferie'] == 0),
        'produits_jardinage'
    ].clip(upper=max_ventes_jour_normal)

    df.loc[
        (df['Weekend'] == 1) | (df['jour_ferie'] == 1),
        'produits_jardinage'
    ] = df.loc[
        (df['Weekend'] == 1) | (df['jour_ferie'] == 1),
        'produits_jardinage'
    ].clip(upper=max_ventes_jour_special)

    return df

def ajouter_bruit_jardinage(df, niveau_bruit=0.1):
    bruit = np.random.normal(loc=0.0, scale=niveau_bruit, size=len(df))
    df['produits_jardinage'] = df['produits_jardinage'] * (1 + bruit)
    df['produits_jardinage'] = df['produits_jardinage'].clip(lower=0)  
    df["produits_jardinage"] = df["produits_jardinage"].round().astype(int)
    
    return df
generer_ventes_jardinage_base(df, base_ventes=5)
ajuster_ventes_jardinage(df)
ajouter_bruit_jardinage(df,0.1)


# Modélisation des Ventes d'Ustensiles Jetables: Approche Saisonnière et Événementielle

Ce module génère des données synthétiques réalistes pour les ventes d'ustensiles jetables (assiettes, couverts, gobelets) en intégrant les facteurs météorologiques, saisonniers et comportementaux.

---

## 1. Saisonnalité Marquée

**Coefficients mensuels** :
- **Pic estival** : Juillet (coefficient 2.7) et juin (2.5) avec la saison des barbecues et pique-niques
- **Rebond hivernal** : Décembre (0.3) pour les fêtes de fin d'année
- **Creux hivernal** : Janvier-Février (0.2-0.3) avec la baisse des activités extérieures



---

## 2. Impact des Conditions Météorologiques

**Paramètres clés** :
- **Température optimale** : >25°C (coefficient 1.1 à 1.3)
- **Précipitations** : Réduction jusqu'à -70% par forte pluie
- **Vent** : Réduction progressive à partir de 15 km/h
- **Humidité** : Effet négatif au-delà de 60% d'humidité

**Modélisation** :
```python
if temp_jour >= 25:
    temp_effect = 1.1 + 0.02*(temp_jour-25)  # Croissance avec la chaleur
elif temp_jour < 10:
    temp_effect = 0.4  # Réduction importante par temps froid
```
## 3. Facteurs Comportementaux

### Jours à fort impact

- **Weekends**  
  Coefficient : 1.8 (+80%)  
  Justification : Augmentation significative pour les repas en famille et les activités sociales

- **Jours fériés**  
  Coefficient : 1.5 (+50%)  
  Justification : Hausse des rassemblements et événements festifs

- **Vacances scolaires**  
  Coefficient : 1.6 (+60%)  
  Justification : Augmentation des activités familiales et sorties

### Effets géographiques

- **Zones côtières**  
  Coefficient : 1.7 (+70%)  
  Explication : Forte fréquentation touristique et activités extérieures accrues

- **Grandes villes**  
  Coefficient : 1.3 (+30%)  
  Explication : Densité de population élevée compensant les espaces extérieurs limités

In [ ]:
import numpy as np

def meteo_ustensiles_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):

    # Effet température (les températures chaudes favorisent les activités extérieures)
    if temp_jour < 10:
        temp_effect = 0.4  # Faible activité par temps froid
    elif 10 <= temp_jour < 18:
        temp_effect = 0.4 + 0.05 * (temp_jour - 10)  # Augmentation progressive
    elif 18 <= temp_jour < 25:
        temp_effect = 0.8 + 0.04 * (temp_jour - 18)  # Zone de forte augmentation
    elif 25 <= temp_jour < 35:
        temp_effect = 1.1 + 0.02 * (temp_jour - 25)  # Zone optimale - pic d'activités extérieures
    else:
        temp_effect = 1.3  # Plafond pour les journées très chaudes

    # Effet précipitations (réduisent fortement les activités extérieures)
    if precip_jour == 0:
        precip_effect = 1.0  # Temps sec idéal
    elif 0 < precip_jour <= 2:
        precip_effect = 1.0 - 0.1 * precip_jour  # Légère pluie réduit un peu
    elif 2 < precip_jour <= 10:
        precip_effect = 0.8 - 0.05 * (precip_jour - 2)  # Réduction progressive
    else:
        precip_effect = 0.3  # Forte pluie réduit significativement les activités extérieures

    # Effet vent (réduit légèrement le confort des activités extérieures)
    if wind_speed_jour < 15:
        wind_effect = 1.0  # Pas d'effet avec vent faible à modéré
    elif 15 <= wind_speed_jour < 30:
        wind_effect = 1.0 - 0.01 * (wind_speed_jour - 15)  # Réduction légère
    else:
        wind_effect = 0.85 - 0.02 * (wind_speed_jour - 30)  # Réduction plus importante
        wind_effect = max(0.6, wind_effect)  # Plancher à 60%

    # Effet humidité (affecte le confort des activités extérieures)
    if humidity_jour < 60:
        humidity_effect = 1.0  # Humidité confortable
    elif 60 <= humidity_jour < 80:
        humidity_effect = 1.0 - 0.005 * (humidity_jour - 60)  # Réduction légère
    else:
        humidity_effect = 0.9 - 0.01 * (humidity_jour - 80)  # Réduction plus importante
        humidity_effect = max(0.7, humidity_effect)  # Plancher à 70%

    # Combinaison des effets
    return max(0.2, temp_effect * precip_effect * wind_effect * humidity_effect)

def calcul_impact_climatique_ustensiles(temp_jour, humidity_jour, precip_jour, wind_speed_jour):

    impact = meteo_ustensiles_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)
    return impact

def generer_ventes_ustensiles_base(df, base_ventes=50):


    df['ustensiles_jetables'] = df.apply(
        lambda row: calcul_impact_climatique_ustensiles(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

# Définition des coefficients mensuels pour ustensiles jetables (pic en été)
mois_ustensiles_coefficients = {
    1: 0.2,  # Janvier - Hiver
    2: 0.3,  # Février - Fin d'hiver
    3: 0.5,  # Mars - Début du printemps
    4: 0.8,  # Avril - Printemps
    5: 1.8,  # Mai - Début des festivités extérieures
    6: 2.5,  # Juin - Plein été, saison des barbecues et pique-niques
    7: 2.7,  # Juillet - Pic de la saison (vacances d'été)
    8: 2.3,  # Août - Haute saison estivale
    9: 1.4,  # Septembre - Fin d'été
    10: 0.8, # Octobre - Automne
    11: 0.4, # Novembre - Début d'hiver
    12: 0.3  # Décembre - Période de fêtes, léger rebond
}

# Coefficients par type de région
region_ustensiles_coefficients = {
    'Côtière': 1.7,    # Zones côtières: beaucoup d'activités extérieures, tourisme
    'Intérieure': 1.0,  # Zones intérieures: base standard
    'Hors côtier': 0.9}    # Zones urbaines: moins d'activités extérieures mais plus de densité


# Coefficients pour jours spéciaux
special_day_ustensiles_coefficients = {
    'jour_ferie': 1.5,         # Les jours fériés favorisent les rassemblements
    'Weekend': 1.8,            # Les weekends sont propices aux barbecues
    'vacances_scolaires': 1.6, # Période de vacances augmente les activités extérieures
    'ramadan': 1.0,            # pas d'impact
    'new_year': 1.0
}

# Seuils maximaux
max_ventes_jour_normal = 15
max_ventes_jour_special = 20


def ajuster_ventes_ustensiles(df):


    # Coefficient mensuel
    df['mois_coeff'] = df['mois'].map(mois_ustensiles_coefficients)

    # Coefficients régionaux
    df['region_coeff'] = df['profil_cotier'].map(region_ustensiles_coefficients)

    # Bonus pour les zones touristiques en haute saison
    if 'saison_toristique' in df.columns:
        df.loc[(df['saison_toristique'] == 'Haute') & ((df['mois'] >= 6) & (df['mois'] <= 8)), 'region_coeff'] *= 1.5

    # Coefficients pour jours spéciaux
    df['special_coeff'] = 1.0


    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
        if day_type in df.columns:
            mask = df[day_type] == 1
            if day_type in special_day_ustensiles_coefficients and mask.any():
                df.loc[mask, 'special_coeff'] = special_day_ustensiles_coefficients[day_type]


    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            df.loc[both_mask, 'special_coeff'] = max(
                special_day_ustensiles_coefficients['jour_ferie'],
                special_day_ustensiles_coefficients['Weekend']
            ) * 1.2

    # Effet des séquences météo favorables (plusieurs jours consécutifs de beau temps)
    df['seq_effect'] = 1.0
    if all(col in df.columns for col in ['temp_jour_lag1', 'temp_jour_lag3', 'precip_jour_lag1', 'precip_jour_lag3']):
        sequence_favorable = (
            (df['temp_jour'] >= 25) &
            (df['temp_jour_lag1'] >= 25) &
            (df['temp_jour_lag3'] >= 25) &
            (df['precip_jour'] < 2) &
            (df['precip_jour_lag1'] < 2) &
            (df['precip_jour_lag3'] < 2)
        )
        df.loc[sequence_favorable, 'seq_effect'] = 1.3  # +30% après 4jours de beau temps

    # Effet de la densité urbaine
    densite_map = {
        'Grande métropole': 1.3,  # Plus de gens = plus d'événements sociaux
        'Ville moyenne': 1.1,     # Bon niveau d'activités sociales
        'Petite ville': 0.9     # Moins d'événements de grande ampleur
    }
    if 'densite_urbaine' in df.columns:
        df['densite_effect'] = df['densite_urbaine'].map(densite_map)
    else:
        df['densite_effect'] = 1.0

    # Calcul final des ventes
    df['ustensiles_jetables'] = (
        df['ustensiles_jetables'] *
        df['mois_coeff'] *
        df['region_coeff'] *
        df['special_coeff'] *
        df['seq_effect'] *
        df['densite_effect']
    )


    df.loc[
        (df['special_coeff'] == 1.0),
        'ustensiles_jetables'
    ] = df.loc[
        (df['special_coeff'] == 1.0),
        'ustensiles_jetables'
    ].clip(upper=max_ventes_jour_normal)


    df.loc[
        (df['special_coeff'] > 1.0),
        'ustensiles_jetables'
    ] = df.loc[
        (df['special_coeff'] > 1.0),
        'ustensiles_jetables'
    ].clip(upper=max_ventes_jour_special)
    # Nettoyer les colonnes temporaires utilisées pour les calculs
    cols_to_drop = ['mois_coeff',
        'region_coeff',
        'special_coeff',
        'seq_effect',
        'densite_effect']
    return df.drop(columns=cols_to_drop, errors='ignore')


def ajouter_bruit_ustensiles(df, niveau_bruit=0.12):
    """
    Ajoute un bruit réaliste aux données de ventes d'ustensiles jetables.
    Le niveau de bruit est légèrement plus élevé que pour les autres car les achats d'ustensiles jetables sont plus impulsifs et variables.
    """
    df_avec_bruit = df.copy()


    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))


    df_avec_bruit["ustensiles_jetables"] = df["ustensiles_jetables"] * facteurs_multiplicatifs


    df_avec_bruit["ustensiles_jetables"] = np.maximum(1, df_avec_bruit["ustensiles_jetables"])

    print(df_avec_bruit[~np.isfinite(df_avec_bruit["ustensiles_jetables"])])

    df_avec_bruit["ustensiles_jetables"] = df_avec_bruit["ustensiles_jetables"].round().astype(int)

    return df_avec_bruit


def print_statistiques_ustensiles(df):

    print("=== STATISTIQUES DES VENTES D'USTENSILES JETABLES ===")
    print(f"Ventes moyennes globales: {df['ustensiles_jetables'].mean():.2f}")


    if 'mois' in df.columns:
        for mois in sorted(df['mois'].unique()):
            print(f"Ventes moyennes au mois {mois}: {df.loc[df['mois'] == mois, 'ustensiles_jetables'].mean():.2f}")


    if 'Weekend' in df.columns:
        print(f"Ventes moyennes pendant Weekend: {df.loc[df['Weekend'] == 1, 'ustensiles_jetables'].mean():.2f}")
        print(f"Ventes moyennes hors weekend: {df.loc[df['Weekend'] == 0, 'ustensiles_jetables'].mean():.2f}")

    if 'jour_ferie' in df.columns:
        print(f"Ventes moyennes pendant jours fériés: {df.loc[df['jour_ferie'] == 1, 'ustensiles_jetables'].mean():.2f}")
        print(f"Ventes moyennes hors jours fériés: {df.loc[df['jour_ferie'] == 0, 'ustensiles_jetables'].mean():.2f}")

    if 'vacances_scolaires' in df.columns:
        print(f"Ventes moyennes pendant vacances scolaires: {df.loc[df['vacances_scolaires'] == 1, 'ustensiles_jetables'].mean():.2f}")
        print(f"Ventes moyennes hors vacances scolaires: {df.loc[df['vacances_scolaires'] == 0, 'ustensiles_jetables'].mean():.2f}")


    if 'profil_cotier' in df.columns:
        for region in df['profil_cotier'].unique():
            print(f"Ventes moyennes en région {region}: {df.loc[df['profil_cotier'] == region, 'ustensiles_jetables'].mean():.2f}")

    print("================================================")


def generer_ventes_ustensiles_completes(df, base_ventes=50, niveau_bruit=0.12):

    df = generer_ventes_ustensiles_base(df, base_ventes=base_ventes)


    df = ajuster_ventes_ustensiles(df)


    df = ajouter_bruit_ustensiles(df, niveau_bruit=niveau_bruit)


    print_statistiques_ustensiles(df)

    return df


df = generer_ventes_ustensiles_completes(df, base_ventes=10, niveau_bruit=0.12)

Empty DataFrame
Columns: [date, city, profil_cotier, densite_urbaine, region_climatique, temp_jour, humidity_jour, wind_speed_jour, precip_jour, point de rosée/jour, temp_jour_lag1, humidity_jour_lag1, wind_speed_jour_lag1, precip_jour_lag1, temp_jour_lag3, humidity_jour_lag3, wind_speed_jour_lag3, precip_jour_lag3, temp_jour_lag7, humidity_jour_lag7, wind_speed_jour_lag7, precip_jour_lag7, mois, Jour, Weekend, jour_ferie, saison, vacances_scolaires, ramadan, new_year, saison_toristique, Indice de Chaleur, refroidissement éolien, Indice d’inconfort, Indice de Vague de Chaleur, Indice de Vague de Froid , Indice de Pluie Intense, Indice de Tempête, Indice de sécheresse, Indice climatique thermique universel (ICTU), boissons_fraiches, snacks_sucré, snacks_salé, produits_laitiers_frais(yaourts, fromage blanc), produits_laitiers_fromage, produits_laitiers(lait,boissons lactées), produits_laitiers(lait glacé,crème glacée), boissons_chaudes, produits anti_moustiques, produits_jardinage, trend

# Modélisation des Ventes de Crème Solaire:Approche Climatique et Démographique

Ce module génère des données synthétiques réalistes pour les ventes de crème solaire en intégrant les facteurs météorologiques et géographiques.

---

## 1. Impact des Conditions Météorologiques

### Effet de la Température
- **<15°C** : Coefficient 0.3 (ventes très faibles)
- **15-25°C** : Augmentation progressive de 0.5 à 0.8
- **>25°C** : Forte augmentation (jusqu'à +50% avec coefficient max de 1.5)

```python
if temp_jour < 15:
    temp_effect = 0.3
elif 15 <= temp_jour < 25:
    temp_effect = 0.5 + 0.03 * (temp_jour - 15)
else:
    temp_effect = 0.8 + 0.05 * (temp_jour - 25)
```
## Effet de l'Humidité

- **Impact** :  
  Réduction de 0.2% par point d'humidité au-dessus de 60%  
  *Exemple* : À 70% d'humidité → -2% des ventes (10 points × 0.2%)

- **Plancher garanti** :  
  Minimum de 30% des ventes de base conservé quelles que soient les conditions

---

## 2. Facteurs Géographiques

### Densité Urbaine

| Type de Zone       | Coefficient | Explication                     |
|--------------------|-------------|---------------------------------|
| Grande métropole   | 1.3         | +30% (population dense)        |
| Ville moyenne      | 1.2         | +20%                           |
| Petite ville       | 1.0         | Valeur de référence            |

---

## 3. Contrôles et Validation

### Limites des Ventes
- **Seuil maximal** : 80 unités/jour  
  *Pour éviter des valeurs irréalistes*

### Modélisation du Réalisme
- **Bruit aléatoire** : ±10%  
  *Simule les variations naturelles du marché*
---

In [ ]:

max_ventes_jour_normal = 10
max_ventes_jour_special = 18

def meteo_creme_solaire_impact(temp_jour, humidity_jour):

    # Impact de la température
    if temp_jour < 15:
        temp_effect = 0.3
    elif 15 <= temp_jour < 25:
        temp_effect = 0.5 + 0.03 * (temp_jour - 15)
    else:
        temp_effect = 0.8 + 0.05 * (temp_jour - 25)
        temp_effect = min(temp_effect, 1.5)  # Plafonné à +50%

    # Effet humidité (réduit légèrement les ventes)
    humidity_effect = 1 - 0.002 * max(0, humidity_jour - 60)

    return max(0.3, temp_effect * humidity_effect)

def calcul_impact_climatique(temp_jour, humidity_jour):

    impact = meteo_creme_solaire_impact(temp_jour, humidity_jour)
    return impact

def generer_ventes_creme_solaire(df, base_ventes=10):

    df['creme_solaire'] = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour']

        ) * base_ventes ,
        axis=1
    )
    return df

def ajuster_ventes(df):


    # Effet de la densité urbaine
    cotier_map = {
    'Côtière': 1.3,      # 30% de ventes en plus en zones côtières (plages, tourisme, etc.)
    'Hors côtier': 1.1,  # 10% de ventes en plus en zones proches de loisirs extérieurs
    'Intérieure': 0.9,   # Moins de ventes dans les zones éloignées des plages ou montagnes
}

    df['profil_effect'] = df['profil_cotier'].map(cotier_map)

    # Calcul final des ventes
    df['creme_solaire'] = df['creme_solaire']  * df['profil_effect']

    # Limitation des ventes journalières
    df.loc[df['creme_solaire'] > max_ventes_jour_normal, 'creme_solaire'] = max_ventes_jour_normal

    return df

def ajouter_bruit(df, niveau_bruit=0.1):

    df_avec_bruit = df.copy()


    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))


    df_avec_bruit["creme_solaire"] = df["creme_solaire"] * facteurs_multiplicatifs


    df_avec_bruit["creme_solaire"] = np.maximum(1, df_avec_bruit["creme_solaire"])
    df_avec_bruit.loc[df_avec_bruit["creme_solaire"] <2, "creme_solaire"] = 0



    df_avec_bruit["creme_solaire"] = df_avec_bruit["creme_solaire"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes=10, niveau_bruit=0.1):



    df = generer_ventes_creme_solaire(df, base_ventes=base_ventes)


    df = ajuster_ventes(df)


    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)


    print_statistiques(df)

    return df

def print_statistiques(df):

    print("=== STATISTIQUES DES VENTES DE CRÈME SOLAIRE ===")
    print(f"Ventes moyennes globales: {df['creme_solaire'].mean():.2f}")

    for day_type in ['Weekend', 'jour_ferie', 'vacances_scolaires']:
        if day_type in df.columns:
            print(f"Ventes moyennes pendant {day_type}: {df.loc[df[day_type] == 1, 'creme_solaire'].mean():.2f}")
            print(f"Ventes moyennes hors {day_type}: {df.loc[df[day_type] == 0, 'creme_solaire'].mean():.2f}")


    if 'profil_cotier' in df.columns:
        for region in df['profil_cotier'].unique():
            print(f"Ventes moyennes en région {region}: {df.loc[df['profil_cotier'] == region, 'creme_solaire'].mean():.2f}")

    print("===============================")

df = processus_de_generation_complet(df, base_ventes=10, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE CRÈME SOLAIRE ===
Ventes moyennes globales: 38.26
Ventes moyennes pendant Weekend: 38.25
Ventes moyennes hors Weekend: 38.26
Ventes moyennes pendant jour_ferie: 43.93
Ventes moyennes hors jour_ferie: 37.98
Ventes moyennes pendant vacances_scolaires: 46.77
Ventes moyennes hors vacances_scolaires: 35.08
Ventes moyennes en région Hors côtier: 41.88
Ventes moyennes en région Intérieure: 35.56
Ventes moyennes en région Côtière: 40.91


In [ ]:
df= df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data1.csv')
print(df.shape)

(43769, 52)


# Modélisation des Ventes d'Équipement d'Urgence: Approche Basée sur les Conditions Extrêmes

Ce module génère des données réalistes pour les ventes de produits d'urgence (bougies, lampes de poche, piles) en fonction des conditions météorologiques critiques.

---

## 1. Impact des Conditions Météorologiques

### Effet de la Température
| Conditions Thermiques | Coefficient | Impact |
|-----------------------|-------------|--------|
| Températures extrêmes (<0°C ou >35°C) | 1.3 | +30% |
| Températures limites (<10°C ou >30°C) | 1.1 | +10% |
| Températures normales | 1.0 | Neutre |

### Effet des Précipitations
| Niveau de Pluie | Coefficient | Impact |
|-----------------|-------------|--------|
| >60mm | 1.4 | +40% |
| 30-60mm | 1.2 | +20% |
| <30mm | 1.0 | Neutre |

### Effet du Vent
| Vitesse du Vent | Coefficient | Impact |
|-----------------|-------------|--------|
| >30 km/h | 1.3 | +30% |
| 15-30 km/h | 1.15 | +15% |
| <15 km/h | 1.0 | Neutre |

**Plafond global** : +100% (coefficient max de 2.0)

---

## 2. Facteurs Démographiques

### Densité Urbaine
| Type de Zone | Coefficient | Explication |
|--------------|-------------|-------------|
| Grande métropole | 1.3 | +30% (population dense) |
| Ville moyenne | 1.2 | +20% |
| Petite ville | 1.0 | Référence |

---

## 3. Paramètres de Contrôle

- **Ventes de base** : 20 unités/jour
- **Maximum journalier** : 100 unités
- **Bruit aléatoire** : ±10% pour le réalisme

---

In [ ]:
'''Équipement d'urgence(Bougies, lampes de poche ,Piles, batteries rechargeables...)'''

def meteo_equipement_impact(temp_jour, precip_jour, wind_speed_jour):
    """
    Les conditions météorologiques extrêmes augmentent la vente d'équipement.
    """
    # Impact de la température
    if temp_jour < 0 or temp_jour > 35:
        temp_effect = 1.3  # +30% en cas de températures extrêmes
    elif temp_jour < 10 or temp_jour > 30:
        temp_effect = 1.1  # +10% en cas de températures limites
    else:
        temp_effect = 1.0  # Impact neutre

    # Impact des précipitations
    if precip_jour > 60:  # Précipitations importantes
        precip_effect = 1.4  # +40%
    elif precip_jour > 30:
        precip_effect = 1.2  # +20%
    else:
        precip_effect = 1.0  # Neutre

    # Impact du vent
    if wind_speed_jour > 30:  # Vent fort
        wind_effect = 1.3  # +30%
    elif wind_speed_jour > 15:
        wind_effect = 1.15  # +15%
    else:
        wind_effect = 1.0  # Neutre

    return min(2.0, temp_effect * precip_effect * wind_effect)  # Plafonné à +100%

def calcul_impact_climatique(temp_jour, precip_jour, wind_speed_jour):

    return meteo_equipement_impact(temp_jour, precip_jour, wind_speed_jour)

def generer_ventes_equipement_urgence(df, base_ventes=10):

    df['equipement_urgence'] = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df
cotier_map = {
    'Côtière': 1.2,
    'Hors côtier': 1.1,
    'Intérieure': 1.4,
}
def ajuster_ventes(df):
    df['cotier_effect'] = df['profil_cotier'].map(cotier_map)
    df['equipement_urgence'] = df['equipement_urgence'] * df['cotier_effect']
    max_ventes_jour = 8  # Légèrement augmenté
    df['equipement_urgence'] = df['equipement_urgence'].clip(upper=max_ventes_jour)
    return df

def ajouter_bruit(df, niveau_bruit=0.1):

    df_avec_bruit = df.copy()


    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))


    df_avec_bruit["equipement_urgence"] = df["equipement_urgence"] * facteurs_multiplicatifs


    df_avec_bruit["equipement_urgence"] = np.maximum(1, df_avec_bruit["equipement_urgence"])
    df_avec_bruit.loc[df_avec_bruit["equipement_urgence"] < 3, "equipement_urgence"] = 0


    df_avec_bruit["equipement_urgence"] = df_avec_bruit["equipement_urgence"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes=5, niveau_bruit=0.1):


    df = generer_ventes_equipement_urgence(df, base_ventes=base_ventes)


    df = ajuster_ventes(df)


    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)



    return df




df = processus_de_generation_complet(df, base_ventes=5, niveau_bruit=0.15)
print(df['equipement_urgence'].unique())
if 'profil_cotier' in df.columns:
        for region in df['profil_cotier'].unique():
            print(f"Ventes moyennes en région {region}: {df.loc[df['profil_cotier'] == region, 'equipement_urgence'].mean():.2f}")

[25 18 24 26 22 19 20 21 23 28 17 16 29 27 31 15 30 33 34 32 35 36 38 37
 13 14 43 39 41 12 42 49 44 46 40 45 50 54 51 57 47 52 48 55]


In [ ]:
df= df.to_csv('data1.csv',index=False)

In [ ]:
df= pd.read_csv('data.csv')

In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
import pandas as pd
df=pd.read_csv('data.csv')

# Modélisation des Ventes de Produits d'Hygiène : Approche Climatique et Saisonnière

Ce module génère des données synthétiques réalistes pour les ventes de produits d'hygiène personnelle (déodorants et lingettes) en fonction des conditions météorologiques et des saisons.

---

## 1. Impact des Conditions Météorologiques

### Effet sur les Déodorants
| Conditions | Coefficient | Impact |
|------------|-------------|--------|
| Température >30°C | 1.4 | +40% |
| Température 25-30°C | 1.2 | +20% |
| Humidité >80% | 1.3 | +30% |
| Humidité 60-80% | 1.2 | +20% |

### Effet sur les Lingettes
| Conditions | Coefficient | Impact |
|------------|-------------|--------|
| Température >30°C | 1.3 | +30% |
| Précipitations >20mm | 1.2 | +20% |

**Plafond global** : +100% (coefficient max de 2.0)

---

## 2. Facteurs Saisonniers

### Coefficient Saison
| Saison | Coefficient | Impact |
|--------|-------------|--------|
| Été | 1.4 | +40% |
| Printemps/Automne | 1.2 | +20% |
| Hiver | 1.0 | Référence |

---

## 3. Paramètres de Contrôle

- **Ventes de base** :
  - Déodorants: 30 unités/jour
  - Lingettes: 25 unités/jour
- **Maximum journalier** :
  - Déodorants: 150 unités
  - Lingettes: 100 unités
- **Bruit aléatoire** : ±10% pour le réalisme

---

In [ ]:
def meteo_hygiene_impact(temp_jour, humidity_jour, precip_jour):
    """
    Calcule l'impact météorologique sur les ventes de produits d'hygiène et soins personnels.

    """
    # Impact de la température
    if temp_jour > 30:  # Chaleur importante
        temp_effect_deodorant = 1.4  # +40% de ventes de déodorants
        temp_effect_lingettes = 1.3  # +30% de lingettes rafraîchissantes
    elif temp_jour > 25:
        temp_effect_deodorant = 1.2  # +20%
        temp_effect_lingettes = 1.2  # +20%
    else:
        temp_effect_deodorant = 1.0
        temp_effect_lingettes = 1.0

    # Impact de l'humidité
    if humidity_jour > 80:  # Humidité très élevée
        humidity_effect = 1.3  # Augmentation des ventes de produits anti-transpiration
    elif humidity_jour > 60:
        humidity_effect = 1.2
    else:
        humidity_effect = 1.0

    # Impact des précipitations
    if precip_jour > 20:  # Précipitations significatives
        precip_effect_lingettes = 1.2  # +20% de lingettes de nettoyage
    else:
        precip_effect_lingettes = 1.0

    # Combinaison des effets
    return {
        'deodorant': min(2.0, temp_effect_deodorant * humidity_effect),
        'lingettes': min(2.0, temp_effect_lingettes * precip_effect_lingettes)
    }

def calcul_impact_climatique(temp_jour, humidity_jour, precip_jour):

    return meteo_hygiene_impact(temp_jour, humidity_jour, precip_jour)

def generer_ventes_hygiene(df, base_ventes_deodorant=30, base_ventes_lingettes=25):

    # Calculer l'impact climatique
    impact_climatique = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour']
        ),
        axis=1
    )

    # Générer les ventes
    df['ventes_deodorant'] = impact_climatique.apply(lambda x: x['deodorant'] * base_ventes_deodorant)
    df['ventes_lingettes'] = impact_climatique.apply(lambda x: x['lingettes'] * base_ventes_lingettes)

    return df

def ajuster_ventes(df):

    # Coefficient saison
    saison_map = {
        'été': 1.4,  # Plus de ventes en été
        'printemps/automne': 1.2,
        'hiver': 1.0
    }
    df['saison_effect'] = df['saison'].map(saison_map)

    # Calcul final des ventes ajustées
    df['ventes_deodorant'] = (
        df['ventes_deodorant'] *
        df['saison_effect']
    )

    df['ventes_lingettes'] = (
        df['ventes_lingettes'] *
        df['saison_effect']
    )


    max_ventes_deodorant = 150
    max_ventes_lingettes = 100

    df['ventes_deodorant'] = df['ventes_deodorant'].clip(upper=max_ventes_deodorant)
    df['ventes_lingettes'] = df['ventes_lingettes'].clip(upper=max_ventes_lingettes)

    return df

def ajouter_bruit(df, niveau_bruit=0.1):

    df_avec_bruit = df.copy()


    bruit_deodorant = np.random.normal(1.0, niveau_bruit, size=len(df))
    bruit_lingettes = np.random.normal(1.0, niveau_bruit, size=len(df))


    df_avec_bruit["ventes_deodorant"] = df["ventes_deodorant"] * bruit_deodorant
    df_avec_bruit["ventes_lingettes"] = df["ventes_lingettes"] * bruit_lingettes


    df_avec_bruit["ventes_deodorant"] = np.maximum(1, df_avec_bruit["ventes_deodorant"])
    df_avec_bruit["ventes_lingettes"] = np.maximum(1, df_avec_bruit["ventes_lingettes"])


    df_avec_bruit["ventes_deodorant"] = df_avec_bruit["ventes_deodorant"].round().astype(int)
    df_avec_bruit["ventes_lingettes"] = df_avec_bruit["ventes_lingettes"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes_deodorant=30, base_ventes_lingettes=25, niveau_bruit=0.1):


    df = generer_ventes_hygiene(df, base_ventes_deodorant, base_ventes_lingettes)


    df = ajuster_ventes(df)


    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)


    print_statistiques(df)

    return df

def print_statistiques(df):

    print("=== STATISTIQUES DES VENTES DE PRODUITS D'HYGIÈNE ===")
    print(f"Ventes moyennes de déodorants: {df['ventes_deodorant'].mean():.2f}")
    print(f"Ventes moyennes de lingettes: {df['ventes_lingettes'].mean():.2f}")



    if 'saison' in df.columns:
        for saison in df['saison'].unique():
            print(f"Ventes moyennes de déodorants pendant {saison}: {df.loc[df['saison'] == saison, 'ventes_deodorant'].mean():.2f}")
            print(f"Ventes moyennes de lingettes pendant {saison}: {df.loc[df['saison'] == saison, 'ventes_lingettes'].mean():.2f}")

    print("===============================")


df = processus_de_generation_complet(df, base_ventes_deodorant=30, base_ventes_lingettes=25, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE PRODUITS D'HYGIÈNE ===
Ventes moyennes de déodorants: 42.13
Ventes moyennes de lingettes: 32.99
Ventes moyennes de déodorants pendant hiver: 34.33
Ventes moyennes de lingettes pendant hiver: 25.77
Ventes moyennes de déodorants pendant printemps/automne: 40.60
Ventes moyennes de lingettes pendant printemps/automne: 31.64
Ventes moyennes de déodorants pendant été: 52.86
Ventes moyennes de lingettes pendant été: 42.78


In [ ]:
print('ventes_deodorant')
print(df['ventes_deodorant'].unique())
print('ventes_lingettes')
print(df['ventes_lingettes'].unique())

ventes_deodorant
[33 27 35 29 28 34 32 26 23 31 25 24 30 37 36 22 38 43 39 42 40 46 44 41
 49 47 48 45 51 53 52 60 50 56 54 58 55 57 59 65 61 62 67 63 66 64 68 70
 69 72 78 77 71 73 21 19 76 74 81 75 20]
ventes_lingettes
[24 29 22 28 25 21 23 27 26 30 33 20 19 31 32 34 35 38 36 37 44 39 40 41
 42 43 50 46 45 49 48 47 54 53 51 55 58 56 52 57 18 16 17 59 60 62 64 15
 61 72 68 65 14]


# Modélisation des Ventes de Produits d'Hydratation : Approche Climatique et Environnementale

Ce module génère des données synthétiques réalistes pour les ventes de produits d'hydratation en fonction des conditions météorologiques.

---

## 1. Impact des Conditions Météorologiques

### Effet de la Température
| Conditions Thermiques | Coefficient | Impact |
|-----------------------|-------------|--------|
| Température <10°C | 1.4 | +40% (peau sèche par froid) |
| Température >30°C | 1.2 | +20% (déshydratation) |
| Température 10-30°C | 1.0 | Neutre |

### Effet de l'Humidité
| Niveau d'Humidité | Coefficient | Impact |
|-------------------|-------------|--------|
| <40% | 1.3 | +30% (air sec) |
| >80% | 0.8 | -20% (air humide) |
| 40-80% | 1.0 | Neutre |

### Effet du Vent
| Vitesse du Vent | Coefficient | Impact |
|-----------------|-------------|--------|
| >20 km/h | 1.3 | +30% (effet desséchant) |
| ≤20 km/h | 1.0 | Neutre |

**Plafond global** : +100% (coefficient max de 2.0)

---

## 2. Paramètres de Contrôle

- **Ventes de base** : 30 unités/jour
- **Maximum journalier** : 100 unités
- **Bruit aléatoire** : ±10% pour le réalisme

---

In [ ]:
def meteo_hydratation_impact(temp_jour, humidity_jour, wind_speed_jour):
    """
    Calcule l'impact météorologique global sur les produits d'hydratation.

    """
    # Impact de la température
    if temp_jour < 10:  # Température froide
        temp_effect = 1.4  # +40% produits hydratants
    elif temp_jour > 30:  # Température chaude
        temp_effect = 1.2  # +20%
    else:
        temp_effect = 1.0

    # Impact de l'humidité
    if humidity_jour < 40:  # Faible humidité
        humidity_effect = 1.3  # Augmentation des ventes de produits hydratants
    elif humidity_jour > 80:  # Humidité très élevée
        humidity_effect = 0.8
    else:
        humidity_effect = 1.0

    # Impact du vent
    if wind_speed_jour > 20:  # Vent fort
        wind_effect = 1.3  # +30% produits de protection
    else:
        wind_effect = 1.0

    # Combinaison des effets
    return min(2.0, temp_effect * humidity_effect * wind_effect)

def calcul_impact_climatique(temp_jour, humidity_jour, wind_speed_jour):

    return meteo_hydratation_impact(temp_jour, humidity_jour, wind_speed_jour)

def generer_ventes_hydratation(df, base_ventes=50):


    impact_climatique = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour'],
            row['wind_speed_jour']
        ),
        axis=1
    )

    # Générer les ventes totales de produits d'hydratation
    df['ventes_produits_hydratation'] = impact_climatique * base_ventes
    # Limitation
    max_ventes = 20
    df['ventes_produits_hydratation'] = df['ventes_produits_hydratation'].clip(upper=max_ventes)

    return df




def ajouter_bruit(df, niveau_bruit=0.1):

    df_avec_bruit = df.copy()


    bruit = np.random.normal(1.0, niveau_bruit, size=len(df))


    df_avec_bruit["ventes_produits_hydratation"] = df["ventes_produits_hydratation"] * bruit


    df_avec_bruit["ventes_produits_hydratation"] = np.maximum(1, df_avec_bruit["ventes_produits_hydratation"])


    df_avec_bruit["ventes_produits_hydratation"] = df_avec_bruit["ventes_produits_hydratation"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes=20, niveau_bruit=0.1):

    df = generer_ventes_hydratation(df, base_ventes)


    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)


    print_statistiques(df)

    return df

def print_statistiques(df):

    print("=== STATISTIQUES DES VENTES DE PRODUITS D'HYDRATATION ===")
    print(f"Ventes moyennes globales: {df['ventes_produits_hydratation'].mean():.2f}")



    if 'saison' in df.columns:
        for saison in df['saison'].unique():
            print(f"Ventes moyennes pendant {saison}: {df.loc[df['saison'] == saison, 'ventes_produits_hydratation'].mean():.2f}")



    print("===============================")


df = processus_de_generation_complet(df, base_ventes=20, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE PRODUITS D'HYDRATATION ===
Ventes moyennes globales: 31.37
Ventes moyennes pendant hiver: 30.99
Ventes moyennes pendant printemps/automne: 30.54
Ventes moyennes pendant été: 33.40


# Modélisation des Ventes de Charbon : Approche Saisonnière et Régionale

Ce module génère des données synthétiques réalistes pour les ventes de charbon en fonction des saisons, des conditions météorologiques et des spécificités régionales.

---

## 1. Impact Saisonniers et Climatiques

### Coefficients Mensuels
| Période | Coefficient | Usage Principal |
|---------|-------------|-----------------|
| Hiver (déc-fév) | 1.7-1.8 | Chauffage |
| Été (juin-août) | 1.2-1.4 | Barbecues |
| Autres mois | 1.0 | - |

### Effet de la Température
| Saison | Conditions | Impact |
|--------|------------|--------|
| Hiver | <10°C | +5% par °C en dessous de 10°C |
| Été (zones côtières) | >25°C | +3% par °C au-dessus de 25°C |

---

## 2. Facteurs Comportementaux

### Weekends et Jours Fériés (Été)
- **Coefficient** : 1.5 (+50%)
- **Explication** : Augmentation des barbecues en période estivale

### Spécificités Régionales
| Type de Zone | Coefficient | Explication |
|--------------|-------------|-------------|
| Côtière | 1.5 | Forte activité touristique estivale |
| Hors côtier | 1.1 | - |
| Intérieure | 0.9 | Moindre usage estival |

---

## 3. Paramètres de Contrôle

- **Ventes de base** : 20 unités/jour
- **Maximum journalier** : 100 unités
- **Bruit aléatoire** : ±10% pour le réalisme
- **Plafond global** : +150% (coefficient max de 2.5)

---

In [ ]:

def meteo_charbon_impact(temp_jour, mois, weekend, jour_ferie,region):
    """
    Calcule l'impact météorologique et saisonnier sur les ventes de charbon.

    """
    # Coefficients saisonniers
    monthly_coeff = {
        1: 1.8, 2: 1.7, 12: 1.8,  # Hiver (chauffage)
        6: 1.2, 7: 1.3, 8: 1.4,   # Été (barbecues)
        **{m: 1.0 for m in [3, 4, 5, 9, 10, 11]}  # Autres mois
    }
    # Coefficients régionaux
    region_coeff = {
    'Côtière': 1.5,   # Zones touristiques (barbecues)
    'Hors côtier':1.1,
    'Intérieure':0.9
}


    # Impact de la température
    if mois in [12, 1, 2]:  # Hiver
        temp_effect = 1.0 + max(0, 0.05 * (10 - temp_jour)) if temp_jour < 10 else 1.0
    elif mois in [6, 7, 8] and region == 'Côtière':  # Été pour zones côtières
        temp_effect = 1.0 + max(0, 0.03 * (temp_jour - 25)) if temp_jour > 25 else 1.0
    else:
        temp_effect = 1.0

    # Impact des weekends et jours fériés
    weekend_holiday_effect = 1.5 if (mois in [6, 7, 8] and (weekend or jour_ferie)) else 1.0
    # Impact des coefficients régionaux
    region_effect = region_coeff.get(region, 1.0)


    # Combinaison des effets
    return min(2.5, monthly_coeff[mois] * temp_effect * weekend_holiday_effect * region_effect)

def calcul_impact_climatique_charbon(temp_jour, mois, weekend, jour_ferie , region):

    return meteo_charbon_impact(temp_jour, mois, weekend, jour_ferie , region)

def generer_ventes_charbon(df, base_ventes=50):


    impact_climatique = df.apply(
        lambda row: calcul_impact_climatique_charbon(
            row['temp_jour'],
            row['mois'],
            row['Weekend'],
            row['jour_ferie'],
            row['profil_cotier']
        ),
        axis=1
    )

    # Générer les ventes totales de charbon
    df['ventes_charbon'] = impact_climatique * base_ventes


    max_ventes = 20
    df['ventes_charbon'] = df['ventes_charbon'].clip(upper=max_ventes)

    return df

def ajouter_bruit(df, niveau_bruit=0.1):

    df_avec_bruit = df.copy()


    bruit = np.random.normal(1.0, niveau_bruit, size=len(df))


    df_avec_bruit["ventes_charbon"] = df["ventes_charbon"] * bruit


    df_avec_bruit["ventes_charbon"] = np.maximum(1, df_avec_bruit["ventes_charbon"])


    df_avec_bruit["ventes_charbon"] = df_avec_bruit["ventes_charbon"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes=20, niveau_bruit=0.1):


    df = generer_ventes_charbon(df, base_ventes)


    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)


    print_statistiques(df)

    return df

def print_statistiques(df):

    print("=== STATISTIQUES DES VENTES DE CHARBON ===")
    print(f"Ventes moyennes globales: {df['ventes_charbon'].mean():.2f}")


    if 'saison' in df.columns:
        for saison in df['saison'].unique():
            print(f"Ventes moyennes pendant {saison}: {df.loc[df['saison'] == saison, 'ventes_charbon'].mean():.2f}")

    print("===============================")


df = processus_de_generation_complet(df, base_ventes=20, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE CHARBON ===
Ventes moyennes globales: 28.32
Ventes moyennes pendant hiver: 37.54
Ventes moyennes pendant printemps/automne: 21.51
Ventes moyennes pendant été: 32.81


In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data1.csv')
print(df.shape)

(43769, 59)


In [ ]:
import pandas as pd
df=pd.read_csv('data1.csv')
print(df.shape)

(43769, 54)


In [ ]:
print(df.columns)

Index(['date', 'city', 'profil_cotier', 'densite_urbaine', 'region_climatique',
       'temp_jour', 'humidity_jour', 'wind_speed_jour', 'precip_jour',
       'point de rosée/jour', 'temp_jour_lag1', 'humidity_jour_lag1',
       'wind_speed_jour_lag1', 'precip_jour_lag1', 'temp_jour_lag3',
       'humidity_jour_lag3', 'wind_speed_jour_lag3', 'precip_jour_lag3',
       'temp_jour_lag7', 'humidity_jour_lag7', 'wind_speed_jour_lag7',
       'precip_jour_lag7', 'mois', 'Jour', 'Weekend', 'jour_ferie', 'saison',
       'vacances_scolaires', 'ramadan', 'new_year', 'saison_toristique',
       'Indice de Chaleur', 'refroidissement éolien', 'Indice d’inconfort',
       'Indice de Vague de Chaleur', 'Indice de Vague de Froid ',
       'Indice de Pluie Intense', 'Indice de Tempête', 'Indice de sécheresse',
       'Indice climatique thermique universel (ICTU)', 'boissons_fraiches',
       'snacks_sucré', 'snacks_salé',
       'produits_laitiers_frais(yaourts, fromage blanc)',
       'produits_laiti

# **Conclusion**

Ce notebook a présenté une modélisation complète des ventes synthétiques de divers produits en fonction des conditions météorologiques et des facteurs saisonniers. Les principaux enseignements sont :

## Synthèse des Résultats

1. **Impact climatique significatif** :  
   Tous les produits analysés montrent une sensibilité marquée aux variations de température, d'humidité et des précipitations, avec des coefficients d'impact allant jusqu'à +50% pour certaines conditions extrêmes.

2. **Saisonnalité prononcée** :  
   Les modèles révèlent des variations saisonnières importantes, particulièrement pour :
   - Les produits de chauffage (hiver)
   - Les articles de plein air (été)
   - Les produits de soin (pics en périodes de transition)

3. **Spécificités régionales** :  
   Les zones côtières et urbaines présentent des comportements d'achat distincts, nécessitant des coefficients adaptés.

